In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/FrancescoZanella/RecSysChallenge2024.git

Cloning into 'RecSysChallenge2024'...
remote: Enumerating objects: 3434, done.
remote: Counting objects: 100% (1044/1044), done.
remote: Compressing objects: 100% (383/383), done.
remote: Total 3434 (delta 701), reused 986 (delta 658), pack-reused 2390
Receiving objects: 100% (3434/3434), 43.22 MiB | 25.94 MiB/s, done.
Resolving deltas: 100% (2196/2196), done.


In [2]:
%cd /kaggle/working/RecSysChallenge2024/src

/kaggle/working/RecSysChallenge2024/src


In [3]:
! pip install --upgrade polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 45.4 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.15
    Uninstalling polars-0.20.15:
      Successfully uninstalled polars-0.20.15


In [4]:
! pip install simsimd scikit-learn==1.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.2/382.2 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import random
import logging
import gc
import tqdm

from ebrec.utils._descriptive_analysis import (
    min_max_impression_time_behaviors, 
    min_max_impression_time_history
)
from ebrec.utils._polars import slice_join_dataframes
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    truncate_history,
)
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_USER_COL
)
from ebrec.evaluation.metrics_protocols import *
from polimi.utils._catboost import (
    _preprocessing_history_trendiness_scores, 
    add_history_trendiness_scores_feature, 
    _preprocessing_mean_delay_features, 
    add_mean_delays_features, 
    _preprocessing_window_features, 
    add_window_features, 
    add_trendiness_feature_categories, 
    _preprocessing_article_endorsement_feature, 
    get_unique_categories,
    add_trendiness_feature,
    _preprocessing_article_endorsement_feature_by_article_and_user,
)

from sklearn.preprocessing import QuantileTransformer, OrdinalEncoder

2024-05-19 12:45:28.380771: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 12:45:28.380920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 12:45:28.555531: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

seed = 42
np.random.seed(seed)
random.seed(seed)
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.random.set_seed(seed)
print(tf.__version__)

2.15.0


# Memory Utils

In [7]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 0.8672


In [8]:
def reduce_polars_df_memory_size(df, set_categorical=True):

    start_mem = df.estimated_size('mb')
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type in [pl.Int16, pl.Int32, pl.Int64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df = df.with_columns(pl.col(col).cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df = df.with_columns(pl.col(col).cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
        elif col_type in [pl.UInt16, pl.UInt32, pl.UInt64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt8))
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt16))
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt32))
        elif col_type == pl.Float64:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df = df.with_columns(pl.col(col).cast(pl.Float32))

    gc.collect()
    end_mem = df.estimated_size('mb')
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# Data import

In [9]:
articles = pl.read_parquet('/kaggle/input/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/kaggle/input/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/kaggle/input/ebnerd_small/train/history.parquet')

behaviors_val = pl.read_parquet('/kaggle/input/ebnerd_small/validation/behaviors.parquet')
history_val = pl.read_parquet('/kaggle/input/ebnerd_small/validation/history.parquet')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 1.128


# Articles Preprocessing

In [10]:
articles = articles.with_columns(
    pl.col('image_ids').list.len().alias('num_images'),
    pl.col('title').str.split(by=' ').list.len().alias('title_len'),
    pl.col('subtitle').str.split(by=' ').list.len().alias('subtitle_len'),
    pl.col('body').str.split(by=' ').list.len().alias('body_len'),
    # very important for tf-idf, otherwise multiple tokens for topics with spaces are built
    pl.col('topics').list.eval(pl.element().str.split(by=' ').list.join('_')),
    pl.col('topics').list.len().alias('num_topics'),
    (pl.col('total_pageviews') / pl.col('total_inviews')).alias('total_pageviews/inviews')
)
articles = reduce_polars_df_memory_size(articles)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
articles.head(2)

Memory usage of dataframe is 58.23 MB
Memory usage after optimization is: 58.00 MB
Decreased by 0.4%

RAM memory GB usage = 1.135


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null


In [11]:
unique_entities = articles.select('entity_groups').explode('entity_groups')['entity_groups'].unique().to_list()
unique_entities = [e for e in unique_entities if e is not None]
unique_entities

['LOC', 'PER', 'ORG', 'PROD', 'MISC', 'EVENT']

In [12]:
# in case of two category_str for the same category, one of the two is picked
unique_categories_df = articles.select(['category', 'category_str']).unique('category').drop_nulls('category')
unique_categories = {
    row['category']: row['category_str'] for row in unique_categories_df.iter_rows(named=True)
}
unique_categories

{539: 'ferie',
 414: 'underholdning',
 2975: 'penge',
 140: 'krimi',
 2504: 'bibliotek',
 572: 'side9',
 2737: 'podcast',
 561: 'haandvaerkeren',
 457: 'forbrug',
 498: 'musik',
 22: 'horoskoper',
 2889: 'auto',
 2731: 'dagsorden',
 2341: 'plus',
 1505: 'vin',
 118: 'nyheder',
 806: 'om_ekstra_bladet',
 529: 'biler',
 2077: 'opinionen',
 731: 'services',
 2: 'incoming',
 142: 'sport',
 512: 'nationen',
 68: 'video',
 565: 'sex_og_samliv'}

Mapping sentiment labels to -1, 0, 1 to save memory in the history dataframe.
Taking only article_default type and mapping it to 0/1 since after only this is needed (and it saves memory) for the history at the moment

In [13]:
articles = articles.with_columns(
    pl.when(pl.col('sentiment_label') == 'Negative').then(-1) \
        .otherwise(
            pl.when(pl.col('sentiment_label') == 'Positive').then(1).otherwise(0)
        ).cast(pl.Int8).alias('sentiment_label_int'),
    (pl.col('article_type') == 'article_default').cast(pl.UInt8).alias('is_article_default')
)

## Topics TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
articles = articles.with_columns(
    pl.Series(
        vectorizer.fit_transform(
            articles.with_columns(pl.col('topics').list.join(separator=' '))['topics'].to_list()
        ).toarray().astype(np.float32)
    ).alias('topics_idf')
)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
articles.head(2)


RAM memory GB usage = 1.151


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews,sentiment_label_int,is_article_default,topics_idf
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32,i8,u8,list[f32]
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null,-1,1,"[0.0, 0.0, … 0.0]"
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null,1,1,"[0.0, 0.0, … 0.536059]"


In [15]:
# without split + join with _ of the single topics this returns 94
len(vectorizer.get_feature_names_out())

79

## Topic Model

In [16]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

stopwords = nltk.corpus.stopwords.words('danish')
title_vectorizer = CountVectorizer(stop_words=stopwords)

titles_bow = title_vectorizer.fit_transform(articles['title'].to_list())

In [17]:
%%time

from sklearn.decomposition import LatentDirichletAllocation

# parameters tuned on perplexity
N_COMPONENTS = 5
lda_model = LatentDirichletAllocation(
    n_components=N_COMPONENTS,
    doc_topic_prior=0.99,
    topic_word_prior=0.75,
    learning_method='online'
)
articles = articles.with_columns(
    pl.Series(lda_model.fit_transform(titles_bow).astype(np.float32)) \
        .list.to_array(5).alias('topic_model_embeddings')
)
articles.head(2)

CPU times: user 37.9 s, sys: 0 ns, total: 37.9 s
Wall time: 38 s


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews,sentiment_label_int,is_article_default,topics_idf,topic_model_embeddings
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32,i8,u8,list[f32],"array[f32, 5]"
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null,-1,1,"[0.0, 0.0, … 0.0]","[0.143627, 0.143834, … 0.143573]"
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null,1,1,"[0.0, 0.0, … 0.536059]","[0.243209, 0.193003, … 0.110556]"


In [18]:
topic_model_columns = [f'topic_model_embedding_{i}' for i in range(N_COMPONENTS)]

articles = articles.with_columns(
    pl.col('topic_model_embeddings').arr.to_struct(fields=lambda i: f'topic_model_embedding_{i}')
).with_columns([
    pl.col('topic_model_embeddings').struct.field(f'topic_model_embedding_{i}') for i in range(N_COMPONENTS)
]).drop('topic_model_embeddings')

articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews,sentiment_label_int,is_article_default,topics_idf,topic_model_embedding_0,topic_model_embedding_1,topic_model_embedding_2,topic_model_embedding_3,topic_model_embedding_4
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32,i8,u8,list[f32],f32,f32,f32,f32,f32
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null,-1,1,"[0.0, 0.0, … 0.0]",0.143627,0.143834,0.287673,0.281293,0.143573
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null,1,1,"[0.0, 0.0, … 0.536059]",0.243209,0.193003,0.336253,0.116979,0.110556


# History preprocessing

In [19]:
def add_topics_tf_idf_columns(df, topics_col, vectorizer):
    return df.with_columns(
        pl.Series(
            vectorizer.transform(
                df.with_columns(pl.col(topics_col).list.join(separator=' '))[topics_col].to_list()
            ).toarray().astype(np.float32)
        ).alias(f'{topics_col}_tf_idf')
    )

In [20]:
%%time

STRONG_THR = 0.8

history_train = pl.concat(
    rows.with_columns(pl.col('article_id_fixed').list.len().alias('NumArticlesHistory')) \
        .explode(['article_id_fixed', 'impression_time_fixed', 'read_time_fixed', 'scroll_percentage_fixed']) \
        .sort(by=['user_id', 'impression_time_fixed']) \
        .with_columns(
            pl.col('impression_time_fixed').dt.weekday().alias('weekday'),
            pl.col('impression_time_fixed').dt.hour().alias('hour'),
        ).join(articles.select(['article_id', 'category', 'is_article_default', 'sentiment_label_int', 
                                'sentiment_score', 'entity_groups', 'topics']), 
               left_on='article_id_fixed', right_on='article_id', how='left') \
        .with_columns(
            (pl.col('sentiment_label_int') == 0).alias('is_neutral'),
            (pl.col('sentiment_label_int') == 1).alias('is_positive'),
            (pl.col('sentiment_label_int') == -1).alias('is_negative'),
            ((pl.col('sentiment_label_int') == 0) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_neutral'),
            ((pl.col('sentiment_label_int') == 1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_positive'),
            ((pl.col('sentiment_label_int') == -1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_negative'),
            pl.col('entity_groups').list.unique(),
        ).group_by('user_id').agg(
            pl.col('article_id_fixed'),
            pl.col('impression_time_fixed'),
            pl.col('category'),
            pl.col('NumArticlesHistory').first(),
            pl.col('read_time_fixed').median().alias('MedianReadTime'),
            pl.col('read_time_fixed').max().alias('MaxReadTime'),
            pl.col('read_time_fixed').sum().alias('TotalReadTime'),
            pl.col('scroll_percentage_fixed').median().alias('MedianScrollPercentage'),
            pl.col('scroll_percentage_fixed').max().alias('MaxScrollPercentage'),
            (pl.col('is_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('NeutralPct'),
            (pl.col('is_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PositivePct'),
            (pl.col('is_negative').sum() / pl.col('NumArticlesHistory').first()).alias('NegativePct'),
            (pl.col('strong_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNeutral'),
            (pl.col('strong_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongPositive'),
            (pl.col('strong_negative').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNegative'),
            (1 - (pl.col('is_article_default').sum() / pl.col('NumArticlesHistory').first())).alias('PctNotDefaultArticles'),
            pl.col('category').mode().alias('MostFrequentCategory'),
            pl.col('weekday').mode().alias('MostFrequentWeekday'),
            pl.col('hour').mode().alias('MostFrequentHour'),
            pl.col('entity_groups').flatten(),
            pl.col('topics').flatten().alias('topics_flatten')
        ).pipe(add_topics_tf_idf_columns, topics_col='topics_flatten', vectorizer=vectorizer) \
        .drop('topics_flatten').with_columns(
            pl.col('MostFrequentCategory').list.first(),
            pl.col('MostFrequentWeekday').list.first(),
            pl.col('MostFrequentHour').list.first(),
        ).with_columns(
            (pl.col('entity_groups').list.count_matches(entity) / pl.col('NumArticlesHistory')).alias(f'{entity}Pct')
            for entity in unique_entities
        ).drop('entity_groups')
    for rows in tqdm.tqdm(history_train.iter_slices(1000), total=history_train.shape[0] // 1000)
)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
history_train = reduce_polars_df_memory_size(history_train)
history_train.head(2)

16it [00:17,  1.12s/it]                        



RAM memory GB usage = 1.264
Memory usage of dataframe is 39.38 MB
Memory usage after optimization is: 38.61 MB
Decreased by 2.0%
CPU times: user 20.9 s, sys: 1.81 s, total: 22.8 s
Wall time: 18.6 s


user_id,article_id_fixed,impression_time_fixed,category,NumArticlesHistory,MedianReadTime,MaxReadTime,TotalReadTime,MedianScrollPercentage,MaxScrollPercentage,NeutralPct,PositivePct,NegativePct,PctStrongNeutral,PctStrongPositive,PctStrongNegative,PctNotDefaultArticles,MostFrequentCategory,MostFrequentWeekday,MostFrequentHour,topics_flatten_tf_idf,LOCPct,PERPct,ORGPct,PRODPct,MISCPct,EVENTPct
u32,list[i32],list[datetime[μs]],list[i16],u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i8,i8,list[f32],f32,f32,f32,f32,f32,f32
10623,"[9739035, 9739065, … 9770533]","[2023-04-27 14:29:44, 2023-04-27 14:29:47, … 2023-05-17 17:36:24]","[118, 414, … 140]",143,3.0,1563.0,9623.0,42.5,100.0,0.230769,0.111888,0.657343,0.097902,0.083916,0.608392,0.06993,118,3,16,"[0.016041, 0.062928, … 0.178427]",0.804196,0.874126,0.888112,0.237762,0.223776,0.202797
12016,"[9738216, 9735383, … 9769622]","[2023-04-27 07:39:48, 2023-04-27 07:41:30, … 2023-05-17 23:56:30]","[140, 118, … 140]",264,156.0,1775.0,63672.0,100.0,100.0,0.257576,0.075758,0.666667,0.106061,0.034091,0.602273,0.07197,140,5,20,"[0.009749, 0.045896, … 0.075912]",0.776515,0.787879,0.852273,0.318182,0.117424,0.087121


In [21]:
history_train.shape

(15143, 27)

# Named entity collaborative filtering

In [22]:
user_id_mapping = history_train.sort('user_id').with_row_index() \
    .select(['index', 'user_id']).rename({'index': 'user_index'})

ner_mapping = articles.select(['ner_clusters']) \
    .explode('ner_clusters') \
    .rename({'ner_clusters': 'ner'}) \
    .with_columns(pl.col('ner').str.strip_chars_start('\" ').str.to_lowercase()) \
    .unique('ner').drop_nulls().filter(pl.col('ner').str.len_chars() > 0) \
    .sort('ner').with_row_index().rename({'index': 'ner_index'})

In [23]:
ner_interactions = history_train.select(['user_id', 'article_id_fixed']).explode('article_id_fixed') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_id_fixed', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_id_fixed']) \
    .unique(['user_index', 'ner_index'])

URM_ner = sps.csr_matrix((np.ones(ner_interactions.shape[0]),
                          (ner_interactions['user_index'].to_numpy(), ner_interactions['ner_index'].to_numpy())),
                         shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

ner_interactions_test = behaviors_train.select(['user_id', 'article_ids_clicked']).explode('article_ids_clicked') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_clicked', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_clicked']) \
    .unique(['user_index', 'ner_index'])

URM_ner_test = sps.csr_matrix((np.ones(ner_interactions_test.shape[0]),
                               (ner_interactions_test['user_index'].to_numpy(), ner_interactions_test['ner_index'].to_numpy())),
                              shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

In [24]:
%cd /kaggle/working
! git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi.git

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1519, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1519 (delta 223), reused 220 (delta 212), pack-reused 1264
Receiving objects: 100% (1519/1519), 50.33 MiB | 24.67 MiB/s, done.
Resolving deltas: 100% (909/909), done.


In [25]:
%cd /kaggle/working/RecSys_Course_AT_PoliMi

from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Evaluation.Evaluator import EvaluatorHoldout

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

item_knn = ItemKNNCFRecommender(URM_ner)
item_knn.fit(topK=100, similarity='jaccard')

evaluator = EvaluatorHoldout(URM_ner_test, cutoff_list=[10], exclude_seen=False)
result_df, _ = evaluator.evaluateRecommender(item_knn)
result_df

/kaggle/working/RecSys_Course_AT_PoliMi
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 18909 (43.2%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 43774 (100.0%), 158.30 column/sec. Elapsed time 4.61 min
EvaluatorHoldout: Ignoring 31 ( 0.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 15112 (100.0%) in 45.19 sec. Users per second: 334


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.311971,0.312913,0.030088,0.214433,0.214676,0.469902,0.319612,0.054883,0.766609,0.953152,...,0.997953,0.76504,0.997953,0.002086,5.362616,0.952586,0.016842,0.419597,3.357942,0.009295


In [26]:
pure_svd = PureSVDRecommender(URM_ner)
pure_svd.fit(num_factors=20)

result_df, _ = evaluator.evaluateRecommender(pure_svd)
result_df

PureSVDRecommender: URM Detected 1 ( 0.0%) users with no interactions.
PureSVDRecommender: URM Detected 18909 (43.2%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 5.04 sec
EvaluatorHoldout: Processed 15112 (100.0%) in 23.48 sec. Users per second: 644


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.342628,0.345141,0.048661,0.261759,0.263204,0.670397,0.392258,0.085219,0.858655,1.275579,...,0.997953,0.856898,0.997953,0.001871,5.929536,0.95823,0.015104,0.463956,3.324192,0.009294


In [27]:
%%time

def get_inview_scores(user_ners_df, recommender, f_name):
    user_index = user_ners_df['user_index'].to_list()[0]
    ners = user_ners_df['ner_index'].to_numpy()
    scores = recommender._compute_item_score([user_index], ners)[0, ners]
    return user_ners_df.with_columns(
        pl.Series(scores).alias(f_name)
    )

ner_inview_scores = behaviors_train.select(['user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_inview', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_inview']) \
    .unique(['user_index', 'ner_index']) \
    .sort(['user_index', 'ner_index']).group_by('user_index').map_groups(
        lambda user_df: user_df.pipe(get_inview_scores, recommender=pure_svd, f_name='pure_svd_scores') \
            .pipe(get_inview_scores, recommender=item_knn, f_name='item_knn_scores')
    )

behaviors_ner_scores = behaviors_train.select(['impression_id', 'user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(ner_inview_scores, on=['user_id', 'article_ids_inview'], how='left') \
    .with_columns(pl.col(['pure_svd_scores', 'item_knn_scores']).fill_null(0)) \
    .group_by(['impression_id', 'article_ids_inview']).agg(
        pl.col('pure_svd_scores').sum().alias('total_ner_svd_scores'),
        pl.col('pure_svd_scores').mean().alias('mean_ner_svd_scores'),
        pl.col('pure_svd_scores').max().alias('max_ner_svd_scores'),
        pl.col('item_knn_scores').sum().alias('total_ner_item_knn_scores'),
        pl.col('item_knn_scores').mean().alias('mean_ner_item_knn_scores'),
        pl.col('item_knn_scores').max().alias('max_ner_item_knn_scores'),
    ).with_columns(
        pl.col(['total_ner_svd_scores', 'mean_ner_svd_scores', 
                'total_ner_item_knn_scores', 'mean_ner_item_knn_scores',
                'max_ner_svd_scores', 'max_ner_item_knn_scores']).max().over('impression_id').name.suffix('_max')
    ).with_columns(
        # l1 normalization
        pl.col('total_ner_svd_scores').truediv(pl.col('total_ner_svd_scores_max')),
        pl.col('mean_ner_svd_scores').truediv(pl.col('mean_ner_svd_scores_max')),
        pl.col('max_ner_svd_scores').truediv(pl.col('max_ner_svd_scores_max')),
        pl.col('total_ner_item_knn_scores').truediv(pl.col('total_ner_item_knn_scores_max')),
        pl.col('mean_ner_item_knn_scores').truediv(pl.col('mean_ner_item_knn_scores_max')),
        pl.col('max_ner_item_knn_scores').truediv(pl.col('max_ner_item_knn_scores_max')),
    ).drop(['total_ner_svd_scores_max', 'mean_ner_svd_scores_max', 
            'max_ner_svd_scores_max', 'total_ner_item_knn_scores_max',
            'mean_ner_item_knn_scores_max', 'max_ner_item_knn_scores_max']) \
    .rename({'article_ids_inview': 'article'})
behaviors_ner_scores.head(5)

CPU times: user 4min 40s, sys: 3min 16s, total: 7min 56s
Wall time: 2min 8s


impression_id,article,total_ner_svd_scores,mean_ner_svd_scores,max_ner_svd_scores,total_ner_item_knn_scores,mean_ner_item_knn_scores,max_ner_item_knn_scores
u32,i32,f32,f32,f32,f32,f32,f32
257455855,9774404,0.071022,0.184656,0.39393,0.077804,0.20229,0.473771
473600588,9769135,0.591139,0.540916,0.519077,0.465677,0.305039,0.275845
241078442,9774557,0.134974,0.956162,0.524629,0.275181,1.0,0.839849
123198998,9735909,0.566742,0.647705,0.801443,0.451053,0.515489,0.675142
32779470,9773378,0.166589,0.319271,0.790573,0.14488,0.3395,0.981326


In [28]:
del URM_ner
del URM_ner_test
del ner_inview_scores
del ner_interactions
del ner_interactions_test
gc.collect()

0

# Creating Training Dataframe

In [29]:
def normalize_endorsement(articles_endorsement_raw):
    return articles_endorsement.sort(by='impression_time').with_columns(
        (
            pl.col('endorsement_10h') / 
            pl.col('endorsement_10h').sum().over('impression_time')
        ).alias('normalized_endorsement_10h'),
        (
            pl.col('endorsement_10h') - 
            pl.col('endorsement_10h').rolling_mean(10, min_periods=1).over('article')
        ).alias('endorsement_10h_diff_rolling'),
        (
            pl.col('endorsement_10h').rolling_mean(5, min_periods=1).over('article') - 
            pl.col('endorsement_10h').rolling_mean(10, min_periods=1).over('article')
        ).alias('endorsement_macd'),
        (
            pl.col('endorsement_10h') / 
            pl.col('endorsement_10h').quantile(0.8).over('impression_time')
        ).alias('endorsement_quantile_norm_10h')
    ).with_columns(
        (
            pl.col('normalized_endorsement_10h') / 
            pl.col('normalized_endorsement_10h').rolling_max(10, min_periods=1).over('impression_time')
        ).alias('normalized_endorsement_10h_rolling_max_ratio'),
    )
    

def add_article_endorsement_feature(df_features, normalized_endorsement):
    # old return df_features.join(other=articles_endorsement, on=["article", "impression_time"], how="left")
    normalized_endorsement = normalized_endorsement.with_columns(pl.col('article').cast(pl.Int32))
    return df_features.with_columns(pl.col('impression_time').dt.round('1m').alias('rounded_impression_time'))\
                         .join(normalized_endorsement.rename({'impression_time' : 'rounded_impression_time'}), on=['article','rounded_impression_time'], how='left')\
                         .drop('rounded_impression_time')

In [30]:
def normalize_endorsement_by_article_and_user(articles_endorsement_articleuser_raw):
    return articles_endorsement_articleuser_raw.sort(by=['user_id','impression_time']).with_columns(
        (
            pl.col('endorsement_20h_articleuser') / 
            pl.col('endorsement_20h_articleuser').sum().over(['user_id','impression_time'])
        ).alias('normalized_endorsement_articleuser_20h'),
        (
            pl.col('endorsement_20h_articleuser') - 
            pl.col('endorsement_20h_articleuser').rolling_mean(10, min_periods=1).over(['article','user_id'])
        ).alias('endorsement_20h_articleuser_diff_rolling'),
        (
            pl.col('endorsement_20h_articleuser').rolling_mean(5, min_periods=1).over(['article','user_id']) - 
            pl.col('endorsement_20h_articleuser').rolling_mean(10, min_periods=1).over(['article','user_id'])
        ).alias('endorsement_articleuser_macd'),
        (
            pl.col('endorsement_20h_articleuser') / 
            pl.col('endorsement_20h_articleuser').quantile(0.8).over(['user_id','impression_time'])
        ).alias('endorsement_articleuser_quantile_norm_20h')
    ).with_columns(
        (
            pl.col('endorsement_20h_articleuser') / 
            pl.col('endorsement_20h_articleuser').rolling_max(10, min_periods=1).over(['user_id','impression_time'])
        ).alias('normalized_endorsement_articleuser_20h_rolling_max_ratio'),
    )
    

def add_article_endorsement_feature_by_article_and_user(df_features, normalized_endorsement):
    # old return df_features.join(other=articles_endorsement, on=["article", "impression_time"], how="left")
    normalized_endorsement = normalized_endorsement.with_columns(pl.col('article').cast(pl.Int32))
    return df_features.with_columns(pl.col('impression_time').dt.round('1m').alias('rounded_impression_time'))\
                         .join(normalized_endorsement.rename({'impression_time' : 'rounded_impression_time'}), on=['user_id','article','rounded_impression_time'], how='left')\
                         .drop('rounded_impression_time')

In [31]:
def add_article_endorsement_feature_leak(df_features, normalized_endorsement,period='10h'):
    # old return df_features.join(other=articles_endorsement, on=["article", "impression_time"], how="left")
    normalized_endorsement = normalized_endorsement.with_columns(pl.col('article').cast(pl.Int32)).rename({f'endorsement_{period}':f'endorsement_{period}_leak'}) \
                            .with_columns(
                                pl.col('impression_time') - pl.duration(hours=int(period[:-1]))
                            ).sort('impression_time').set_sorted('impression_time')
    
    return df_features.with_columns(pl.col('impression_time').dt.round('1m').alias('rounded_impression_time'))\
                        .sort('rounded_impression_time').set_sorted('rounded_impression_time') \
                         .join_asof(normalized_endorsement.rename({'impression_time' : 'rounded_impression_time'}),by='article', left_on='rounded_impression_time',right_on='rounded_impression_time', strategy='nearest')\
                         .drop('rounded_impression_time') \
                        .with_columns(
                            pl.col(f'endorsement_{period}_leak').fill_null(0)
                        )


In [32]:
def add_trendiness_feature_leak(df_features: pl.DataFrame, articles: pl.DataFrame, period: str = "3d"):

    topics = articles.select("topics").explode("topics").unique()
    topics = [topic for topic in topics["topics"] if topic is not None]
    # min_impression_time = df_features.select(pl.col("impression_time")).min().item()

    # topics_total_publications= articles.filter(pl.col("published_time")< min_impression_time ).select("topics") \
    # .explode("topics").group_by("topics").len()

    topics_popularity = articles.select(["published_time", "topics"]).with_columns(
        pl.col("published_time").dt.date().alias("published_date")
    ).drop("published_time").group_by("published_date").agg(
        pl.col("topics").flatten()
    ).sort("published_date").set_sorted("published_date").upsample(time_column="published_date", every="1d") \
        .rolling(index_column="published_date", period=period).agg(
        [pl.col("topics").list.count_matches(topic).sum().alias(
            f"{topic}_matches") for topic in topics]
    )

    return df_features.with_columns(
        pl.col("impression_time").dt.date().alias("impression_date")
    ).join(other=articles.select(["article_id", "topics"]), left_on="article", right_on="article_id", how="left") \
        .with_columns(
        [pl.col("topics").list.contains(topic).cast(
            pl.Int8).alias(f"{topic}_present") for topic in topics]
    ).sort('impression_date').set_sorted('impression_date') \
    .join_asof(topics_popularity.with_columns(pl.col('published_date') + pl.duration(days=1) - pl.duration(days=int(period[:-1]))) \
               .sort('published_date').set_sorted('published_date'),
               left_on="impression_date", right_on="published_date", strategy="nearest") \
        .with_columns(
        [pl.col(f"{topic}_present").mul(pl.col(f"{topic}_matches")).alias(
            f"trendiness_score_{topic}") for topic in topics]
    ).with_columns(
        pl.sum_horizontal([pl.col(f"trendiness_score_{topic}") for topic in topics]).alias(
            "trendiness_score_leak"),
    ).drop(
        [f"trendiness_score_{topic}" for topic in topics]
    ).drop(
        [f"{topic}_matches" for topic in topics]
    ).drop(
        [f"{topic}_present" for topic in topics]
    ).drop(["topics", "impression_date","published_date"])

In [33]:
%%time

NPRATIO = 2

users_mean_trendiness_scores, topics_mean_trendiness_scores = _preprocessing_history_trendiness_scores(
        history=history_train, articles=articles)
topic_mean_delays, user_mean_delays = _preprocessing_mean_delay_features(
    articles=articles, history=history_train)
windows, user_windows, user_topics_windows, user_category_windows = _preprocessing_window_features(
    history=history_train, articles=articles)
articles_endorsement = _preprocessing_article_endorsement_feature(
    behaviors=behaviors_train, period="10h")
normalized_endorsement = normalize_endorsement(articles_endorsement)
articles_endorsement_articleuser = _preprocessing_article_endorsement_feature_by_article_and_user(
    behaviors=behaviors_train, period="20h")
normalized_endorsement_articleuser = normalize_endorsement_by_article_and_user(articles_endorsement_articleuser)

topics = articles.select("topics").explode("topics").unique()
topics = [topic for topic in topics["topics"] if topic is not None]

history_counts=history_train.select(['article_id_fixed']).explode('article_id_fixed').group_by('article_id_fixed').len() \
.rename({'article_id_fixed':'article','len':'clicked_count'})

behaviors_counts=behaviors_train.select('article_ids_inview').explode('article_ids_inview').group_by('article_ids_inview').len() \
.rename({'article_ids_inview':'article','len':'inview_count'})

# dropping duplicates that can occur because of sampling strategy
train_ds = behaviors_train.pipe(create_binary_labels_column, shuffle=True, seed=123) \
    .with_columns(pl.col('gender').fill_null(2)) \
    .with_columns(pl.col('postcode').fill_null(5)) \
    .select(['impression_id', 'article_ids_inview', 'impression_time', 'labels', 
             'device_type', 'read_time', 'scroll_percentage', 'user_id', 'is_sso_user', 'gender',
             'age', 'is_subscriber', 'session_id', 'postcode']) \
    .explode(['article_ids_inview', 'labels']) \
    .rename({'article_ids_inview': 'article', 'labels': 'target'}) \
    .with_columns(pl.col('article').cast(pl.Int32)) \
    .pipe(add_trendiness_feature, articles=articles, period='1d') \
    .rename({'trendiness_score': 'trendiness_score_1d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='3d') \
    .rename({'trendiness_score': 'trendiness_score_3d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='5d') \
    .rename({'trendiness_score': 'trendiness_score_5d'}) \
    .pipe(add_trendiness_feature_leak, articles=articles, period='3d') \
    .rename({'trendiness_score_leak':'trendiness_score_3d_leak'}) \
    .unique(['impression_id', 'article']) \
    .with_columns(
        pl.col('impression_time').dt.weekday().alias('weekday'),
        pl.col('impression_time').dt.hour().alias('hour'),
        pl.col('article').cast(pl.Int32),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_3d')
        ).alias('trendiness_score_1d/3d'),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_5d')
        ).alias('trendiness_score_1d/5d'),
        (
            pl.col('trendiness_score_3d') / 
            pl.col('trendiness_score_3d').max().over(pl.col('impression_time').dt.date())
        ).alias('normalized_trendiness_score_overall'),
    ).join(articles.select(['article_id', 'premium', 'published_time', 'category',
                            'sentiment_score', 'sentiment_label', 'entity_groups',
                            'num_images', 'title_len', 'subtitle_len', 'body_len','num_topics',
                            'total_pageviews', 'total_inviews', 'total_read_time', 
                            'total_pageviews/inviews', 'article_type']),
           left_on='article', right_on='article_id', how='left') \
    .with_columns(
        (pl.col('impression_time') - pl.col('published_time')).dt.total_days().alias('article_delay_days'),
        (pl.col('impression_time') - pl.col('published_time')).dt.total_hours().alias('article_delay_hours'),
    ).with_columns(
        pl.col('entity_groups').list.contains(entity).alias(f'Entity_{entity}_Present')
        for entity in unique_entities
    ).drop('entity_groups') \
    .join(behaviors_ner_scores, on=['impression_id', 'article'], how='left') \
    .pipe(add_history_trendiness_scores_feature, articles=articles, 
          users_mean_trendiness_scores=users_mean_trendiness_scores,
          topics_mean_trendiness_scores=topics_mean_trendiness_scores, topics=topics) \
    .pipe(add_mean_delays_features, articles=articles,
          topic_mean_delays=topic_mean_delays, user_mean_delays=user_mean_delays) \
    .pipe(add_window_features, articles=articles, user_windows=user_windows,
          user_category_windows=user_category_windows, 
          user_topics_windows=user_topics_windows, windows=windows) \
    .pipe(add_trendiness_feature_categories, articles=articles) \
    .pipe(add_article_endorsement_feature, normalized_endorsement=normalized_endorsement) \
    .pipe(add_article_endorsement_feature_by_article_and_user, normalized_endorsement = normalized_endorsement_articleuser) \
    .pipe(add_article_endorsement_feature_leak, normalized_endorsement = normalized_endorsement) \
    .rename({
        "endorsement_10h_diff_rolling_right":"endorsement_10h_leak_diff_rolling",
        "endorsement_macd_right":"endorsement_leak_macd",
        "endorsement_quantile_norm_10h_right":"endorsement_quantile_norm_10h_leak",
        "normalized_endorsement_10h_rolling_max_ratio_right":"normalized_endorsement_10h_leak_rolling_max_ratio"
    }).join(history_counts, on='article',how='left') \
    .join(behaviors_counts , on='article', how='left')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)

train_ds = reduce_polars_df_memory_size(train_ds)
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head()

16it [00:27,  1.74s/it]                        
2427it [00:06, 352.48it/s]                          
16it [00:01,  8.10it/s]                        
16it [00:02,  6.60it/s]                        
100%|██████████| 506/506 [00:05<00:00, 90.60it/s] 



RAM memory GB usage = 3.158
Memory usage of dataframe is 1119.91 MB
Memory usage after optimization is: 661.24 MB
Decreased by 41.0%

RAM memory GB usage = 2.68
CPU times: user 12min 24s, sys: 59.1 s, total: 13min 24s
Wall time: 4min 3s


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,session_id,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,published_time,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,…,mean_user_trendiness_score,mean_topics_trendiness_score,mean_topics_mean_delay_days,mean_topics_mean_delay_hours,user_mean_delay_days,user_mean_delay_hours,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count
u32,i32,u32,datetime[μs],i8,i8,f32,f32,bool,i8,i8,bool,u32,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,datetime[μs],i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,…,f32,f32,f32,f32,f32,f32,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16
531181957,9770538,2219289,2023-05-18 07:00:10,1,1,14.0,null,true,0,70,true,220898,5,61,202,355,209,4,7,0.30198,0.171831,0.392233,false,2023-05-18 05:57:20,498,0.6058,"""Negative""",2,10,18,597,6,63862,368843,4.991874e6,0.173141,…,181.252869,282.398804,7.871868,191.826935,49.605366,1193.180054,130,83,75,24,1,0,0,0,3,44,9,16,0.047904,0.0,0.0,2.285714,1.0,1,0.2,0.0,0.0,1.0,1.0,1526,0.014948,4.5,2.5,0.828447,0.765296,554,1792
450331134,9727087,955346,2023-05-18 07:00:08,0,3,325.0,100.0,false,2,null,false,1657450,5,77,217,385,129,4,7,0.354839,0.2,0.421359,true,2023-04-24 09:46:42,118,0.6239,"""Neutral""",5,7,29,721,5,81411,1139178,4.614807e6,0.071465,…,233.199997,364.027435,6.206751,151.412888,0.0,1.616667,24,18,16,9,1,0,0,0,8,12,96,17,0.050898,0.0,0.0,2.428571,1.0,1,0.142857,0.0,0.0,1.0,1.0,2121,0.018005,5.7,3.5,0.923781,0.891176,184,2126
372697938,9768829,904400,2023-05-18 07:00:07,0,1,25.0,null,false,2,null,false,1324242,5,141,409,745,428,4,7,0.344743,0.189262,0.794175,false,2023-05-16 20:40:13,142,0.8115,"""Positive""",3,11,19,463,5,144909,615970,1.1410048e7,0.235253,…,222.654251,390.059143,3.625412,89.494057,8.489362,212.569153,562,0,0,42,1,0,0,0,87,399,110,1,0.002994,0.0,0.0,0.142857,0.066667,1,0.111111,0.0,0.0,1.0,1.0,54,0.000485,-6.8,2.0,0.027328,0.010555,2059,247
338472617,9720068,2269270,2023-05-18 07:00:28,0,2,31.0,null,false,2,null,false,1409226,5,46,105,152,76,4,7,0.438095,0.302632,0.203883,true,2023-04-17 09:27:32,118,0.7884,"""Neutral""",1,7,20,531,2,10821,564767,667781.0,0.01916,…,216.343109,176.763824,9.866658,239.443207,2.739003,68.56012,123,114,96,59,1,0,0,0,58,52,96,7,0.020958,0.0,0.0,1.0,1.0,1,0.111111,0.0,0.0,1.0,1.0,1318,0.014266,-41.0,-1.0,0.633654,0.534036,7,1372
556119040,9770989,49472,2023-05-18 07:00:29,0,2,7.0,null,false,2,null,false,1918426,5,46,105,152,76,4,7,0.438095,0.302632,0.203883,false,2023-05-18 05:36:48,140,0.9948,"""Negative""",3,7,13,141,2,101707,460662,3.551386e6,0.220784,…,252.918365,176.763824,9.866658,239.443207,0.714286,21.367348,26,17,4,18,1,0,0,0,5,6,67,15,0.04491,0.0,0.0,2.142857,1.0,1,0.142857,0.0,0.0,1.0,1.0,1796,0.015621,5.2,2.8,1.099143,0.539663,1029,1942


In [34]:
train_ds.shape

(2585747, 89)

## Session Features

In [35]:
# behaviors_train.select(['session_id', 'impression_time', 'article_ids_clicked']) \
#     .group_by('session_id').map_groups(
#         lambda group: group.explode('article_ids_clicked').sort('impression_time') \
#             .select(
#                 pl.col('session_id').first(),
#                 pl.col('article_ids_clicked').first().alias('article_id'),
#                 (pl.col('impression_time').last() - pl.col('impression_time').first()).dt.total_minutes().alias('induced_session_len'),
#                 (pl.col('impression_time').count() - 1).alias('num_clicked_articles_following')
#             )
#     ).group_by('article_id').agg(
#         pl.col('num_clicked_articles_following').mean().alias('mean_num_clicked_articles_following'),
#         pl.col('induced_session_len').mean().alias('mean_induced_session_len'),
#     )

In [36]:
%%time

last_history_df = history_train.with_columns(
    pl.col('impression_time_fixed').list.max().alias('last_history_impression_time'),
    pl.col('article_id_fixed').list.tail(1).alias('last_history_article'),
).select(['user_id', 'last_history_impression_time', 'last_history_article'])

last_session_time_df = behaviors_train.select(['session_id', 'user_id', 'impression_time', 'article_ids_inview']) \
    .group_by('session_id').agg(
        pl.col('user_id').first(), 
        pl.col('impression_time').max().alias('session_time'), 
        pl.col('article_ids_inview').flatten().alias('all_seen_articles'),
        (pl.col('impression_time').max() - pl.col('impression_time').min()).dt.total_minutes().alias('session_duration'),
    ).with_columns(
        pl.col(['session_time', 'session_duration']).shift(1).over('user_id').name.prefix('last_'),
        pl.col('all_seen_articles').list.unique().shift(1).over('user_id'),
        pl.col('session_duration').rolling_mean(100, min_periods=1).over('user_id').alias('mean_prev_sessions_duration'),
    ).with_columns(pl.col(['last_session_duration']).fill_null(0)) \
    .join(last_history_df, on='user_id', how='left') \
    .with_columns(
        pl.col('last_session_time').fill_null(pl.col('last_history_impression_time')),
        pl.col('all_seen_articles').fill_null(pl.col('last_history_article')),
    ).select(['session_id', 'last_session_time', 'last_session_duration', 
              'all_seen_articles', 'mean_prev_sessions_duration'])
    
gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
last_session_time_df.head(2)


RAM memory GB usage = 2.701
CPU times: user 2.74 s, sys: 508 ms, total: 3.25 s
Wall time: 1.98 s


session_id,last_session_time,last_session_duration,all_seen_articles,mean_prev_sessions_duration
u32,datetime[μs],i64,list[i32],f64
1899220,2023-05-16 21:51:30,0,[9768649],0.0
899882,2023-05-18 06:10:13,0,[9770989],0.0


In [37]:
train_ds = train_ds.join(last_session_time_df, on='session_id', how='left').with_columns(
    (pl.col('impression_time') - pl.col('last_session_time')).dt.total_hours().alias('last_session_time_hour_diff'),
    ((pl.col('last_session_time') - pl.col('published_time')).dt.total_hours() > 0).alias('is_new_article'),
    pl.col('all_seen_articles').list.contains(pl.col('article')).alias('is_already_seen_article'),
).drop(['published_time', 'session_id', 'all_seen_articles', 'last_session_time'])

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)


RAM memory GB usage = 3.048


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,…,user_mean_delay_hours,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article
u32,i32,u32,datetime[μs],i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,…,f32,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16,i64,f64,i64,bool,bool
531181957,9770538,2219289,2023-05-18 07:00:10,1,1,14.0,null,true,0,70,true,5,61,202,355,209,4,7,0.30198,0.171831,0.392233,false,498,0.6058,"""Negative""",2,10,18,597,6,63862,368843,4.991874e6,0.173141,"""article_default""",0,…,1193.180054,130,83,75,24,1,0,0,0,3,44,9,16,0.047904,0.0,0.0,2.285714,1.0,1,0.2,0.0,0.0,1.0,1.0,1526,0.014948,4.5,2.5,0.828447,0.765296,554,1792,8,3.833333,-23,true,false
450331134,9727087,955346,2023-05-18 07:00:08,0,3,325.0,100.0,false,2,null,false,5,77,217,385,129,4,7,0.354839,0.2,0.421359,true,118,0.6239,"""Neutral""",5,7,29,721,5,81411,1139178,4.614807e6,0.071465,"""article_default""",23,…,1.616667,24,18,16,9,1,0,0,0,8,12,96,17,0.050898,0.0,0.0,2.428571,1.0,1,0.142857,0.0,0.0,1.0,1.0,2121,0.018005,5.7,3.5,0.923781,0.891176,184,2126,0,0.0,-32,true,false


In [38]:
train_ds.select(pl.col('is_new_article').sum() / train_ds.shape[0], pl.col('is_already_seen_article').sum() / train_ds.shape[0])

is_new_article,is_already_seen_article
f64,f64
0.559238,0.023812


In [39]:
del last_history_df
del last_session_time_df
gc.collect()

0

## Popularities

In [40]:
%%time

articles_date_popularity = articles.select(['published_time', 'article_id']) \
    .group_by(pl.col('published_time').dt.date().alias('published_date')) \
    .agg(pl.col('article_id').count().alias('daily_articles')) \

published_category_popularity = articles.select(['published_time', 'article_id', 'category']) \
    .group_by([pl.col('published_time').dt.date().alias('published_date'), 'category']) \
    .agg(pl.col('article_id').count().alias('category_daily_articles')) \
    .join(articles_date_popularity, on='published_date', how='left') \
    .with_columns((pl.col('category_daily_articles') / pl.col('daily_articles')).alias('category_daily_pct')) \
    .drop(['category_daily_articles', 'daily_articles'])

train_ds = train_ds.join(published_category_popularity, how='left', right_on=['published_date', 'category'],
                         left_on=[pl.col('impression_time').dt.date() - pl.duration(days=1), 'category']) \
    .rename({'category_daily_pct': 'yesterday_category_daily_pct'}) \
    .with_columns(pl.col('yesterday_category_daily_pct').fill_null(0))
train_ds = reduce_polars_df_memory_size(train_ds)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)

Memory usage of dataframe is 701.62 MB
Memory usage after optimization is: 649.53 MB
Decreased by 7.4%

RAM memory GB usage = 2.748
CPU times: user 2.84 s, sys: 1.56 s, total: 4.4 s
Wall time: 3.93 s


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,…,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article,yesterday_category_daily_pct
u32,i32,u32,date,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,…,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16,i8,f32,i16,bool,bool,f32
531181957,9770538,2219289,2023-05-17,1,1,14.0,null,true,0,70,true,5,61,202,355,209,4,7,0.30198,0.171831,0.392233,false,498,0.6058,"""Negative""",2,10,18,597,6,63862,368843,4.991874e6,0.173141,"""article_default""",0,…,130,83,75,24,1,0,0,0,3,44,9,16,0.047904,0.0,0.0,2.285714,1.0,1,0.2,0.0,0.0,1.0,1.0,1526,0.014948,4.5,2.5,0.828447,0.765296,554,1792,8,3.833333,-23,true,false,0.02381
450331134,9727087,955346,2023-05-17,0,3,325.0,100.0,false,2,null,false,5,77,217,385,129,4,7,0.354839,0.2,0.421359,true,118,0.6239,"""Neutral""",5,7,29,721,5,81411,1139178,4.614807e6,0.071465,"""article_default""",23,…,24,18,16,9,1,0,0,0,8,12,96,17,0.050898,0.0,0.0,2.428571,1.0,1,0.142857,0.0,0.0,1.0,1.0,2121,0.018005,5.7,3.5,0.923781,0.891176,184,2126,0,0.0,-32,true,false,0.222222


# Adding history features

In [41]:
def list_pct_matches_with_col(a, b) -> pl.Expr:
    '''
    Returns an expression to count the percentage of matching element in a list with another column.
    The polars function count_matches cannot be used since it wants only a single element, 
    variable element from row to row.
    '''
    return pl.when(pl.col(a).list.len() == 0).then(0.0) \
        .otherwise((pl.col(a).list.len() - (pl.col(a).list.set_difference(pl.col(b))).list.len()) / pl.col(a).list.len())


def list_pct_matches_with_constant(a, value) -> pl.Expr:
    '''
    Returns an expression to count the percentage of matching element in a list with a constant value.
    The polars function count_matches cannot be used since it wants only a single element, 
    variable element from row to row.
    '''
    return pl.when(pl.col(a).list.len() == 0).then(0.0) \
        .otherwise(pl.col(a).list.count_matches(value) / pl.col(a).list.len())


def cosine_similarity(x, y):
    x = np.array(x)
    y = np.array(y)
    normalization = np.linalg.norm(x, 2) * np.linalg.norm(y, 2)
    return np.dot(x, y) / normalization if normalization > 0 else 0


prev_train_columns = [c for c in train_ds.columns if c not in ['impression_id', 'article', 'impression_time']]

train_ds = pl.concat(
    rows.join(history_train.select(['user_id', 'article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .join(articles.select(['article_id', 'topics', 'entity_groups', 'topics_idf'] + topic_model_columns), 
              left_on='article', right_on='article_id', how='left') \
        .explode(['article_id_fixed', 'impression_time_fixed']) \
        .join(articles.select(['article_id', 'topics', 'entity_groups'] + topic_model_columns), 
              left_on='article_id_fixed', right_on='article_id', how='left') \
        .rename({'topics_right': 'topics_history', 'entity_groups_right': 'entity_groups_history'}) \
        .rename({f'topic_model_embedding_{i}_right': f'topic_model_embedding_{i}_history' for i in range(N_COMPONENTS)}) \
        .with_columns(
            # 1/delay gives the weight for the weighted mean of the lda embeddings
            (1 / (pl.col('impression_time') - pl.col('impression_time_fixed')).dt.total_hours().sqrt()).alias('history_weight'),
            (pl.col("topics").list.set_intersection(pl.col("topics_history")).list.len().truediv(
                pl.col("topics").list.set_union(pl.col("topics_history")).list.len()
            )).alias("JS"),
            pl.col('entity_groups').list.set_intersection(pl.col('entity_groups_history')).list.len().alias('common_entities'),
        ).drop(['entity_groups_history', 'entity_groups', 'topics', 'topics_history']) \
        .with_columns(
            # summing delays to normalize them before weighted mean
            pl.col('history_weight').sum().over(['impression_id', 'article']).alias('history_weight_sum'),
            *[pl.col(x).mul(pl.col(f'{x}_history')).alias(f'{x}_dot') for x in topic_model_columns],
            *[pl.col(x).mul(pl.col(x)) for x in topic_model_columns],
            *[pl.col(f'{x}_history').mul(pl.col(f'{x}_history')).alias(f'{x}_history_square') for x in topic_model_columns],
        ).with_columns(
            # weights now sum to 1
            pl.col('history_weight').truediv(pl.col('history_weight_sum')),
            pl.sum_horizontal(topic_model_columns).sqrt().alias('topic_model_norm'),
            pl.sum_horizontal([f'{x}_history_square' for x in topic_model_columns]).sqrt().alias('topic_model_history_norm'),
            pl.sum_horizontal([f'{x}_dot' for x in topic_model_columns]).alias('topic_model_dot'),
        ).with_columns(
            *[pl.col(f'{x}_history').mul(pl.col('history_weight')).alias(f'{x}_history_weighted') for i, x in enumerate(topic_model_columns)],
            pl.col('topic_model_dot').truediv(pl.col('topic_model_norm').mul('topic_model_history_norm')).alias('topic_model_cosine')
        ).group_by(['impression_id', 'article']).agg(
            pl.col(prev_train_columns).first(),
            pl.col('topics_idf').first(),
            pl.col('common_entities').mean().alias('MeanCommonEntities'),
            pl.col('common_entities').max().alias('MaxCommonEntities'),
            pl.col("JS").mean().alias("mean_JS"),
            pl.col("JS").min().alias("min_JS"),
            pl.col("JS").max().alias("max_JS"),
            pl.col("JS").std().alias("std_JS"),
            pl.col("JS").mul(pl.col("history_weight")).sum().alias("weighted_mean_JS"),
            pl.col("topic_model_cosine").mean().alias("mean_topic_model_cosine"),
            pl.col("topic_model_cosine").min().alias("min_topic_model_cosine"),
            pl.col("topic_model_cosine").max().alias("max_topic_model_cosine"),
            pl.col("topic_model_cosine").std().alias("std_topic_model_cosine"),
            pl.col("topic_model_cosine").mul(pl.col("history_weight")).sum().alias("weighted_mean_topic_model_cosine"),
            *[pl.col(f'{x}_history').mean().alias(f'lda_{i}_history_mean') for i, x in enumerate(topic_model_columns)],
            *[pl.col(f'{x}_history_weighted').sum().alias(f'lda_{i}_history_weighted_mean') for i, x in enumerate(topic_model_columns)]
        ).join(history_train.drop(['article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .with_columns(
            pl.struct(['topics_idf', 'topics_flatten_tf_idf']).map_elements(
                lambda x: cosine_similarity(x['topics_idf'], x['topics_flatten_tf_idf']), return_dtype=pl.Float64
            ).cast(pl.Float32).alias('topics_cosine'),
            (pl.col('category') == pl.col('MostFrequentCategory')).alias('IsFavouriteCategory'),
            pl.col('category_right').list.n_unique().alias('NumberDifferentCategories'),
            list_pct_matches_with_col('category_right', 'category').alias('PctCategoryMatches'),
        ).with_columns(
            [list_pct_matches_with_constant('category_right', c).alias(f'Category_{c_str}_Pct')
             for c, c_str in unique_categories.items()]
        ).drop(['topics_idf', 'topics_flatten', 'topics_flatten_tf_idf', 'category_right'])
    for rows in tqdm.tqdm(train_ds.iter_slices(10000), total=train_ds.shape[0] // 10000)
)
train_ds = reduce_polars_df_memory_size(train_ds)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)

259it [38:50,  9.00s/it]


Memory usage of dataframe is 1700.14 MB
Memory usage after optimization is: 1317.92 MB
Decreased by 22.5%

RAM memory GB usage = 5.02


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,MostFrequentWeekday,MostFrequentHour,LOCPct,PERPct,ORGPct,PRODPct,MISCPct,EVENTPct,topics_cosine,IsFavouriteCategory,NumberDifferentCategories,PctCategoryMatches,Category_ferie_Pct,Category_underholdning_Pct,Category_penge_Pct,Category_krimi_Pct,Category_bibliotek_Pct,Category_side9_Pct,Category_podcast_Pct,Category_haandvaerkeren_Pct,Category_forbrug_Pct,Category_musik_Pct,Category_horoskoper_Pct,Category_auto_Pct,Category_dagsorden_Pct,Category_plus_Pct,Category_vin_Pct,Category_nyheder_Pct,Category_om_ekstra_bladet_Pct,Category_biler_Pct,Category_opinionen_Pct,Category_services_Pct,Category_incoming_Pct,Category_sport_Pct,Category_nationen_Pct,Category_video_Pct,Category_sex_og_samliv_Pct
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,i8,i8,f32,f32,f32,f32,f32,f32,f32,bool,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
68358190,9769553,1190605,0,2,21.0,null,false,2,null,false,5,102,281,386,215,4,7,0.362989,0.264249,0.545631,false,140,0.9262,"""Negative""",2,4,33,448,3,71089,436766,5.159866e6,0.162762,"""article_default""",0,1,…,2,15,0.790528,0.885246,0.934426,0.20765,0.162113,0.245902,0.433752,false,8,0.98725,0.0,0.102004,0.005464,0.176685,0.0,0.0,0.0,0.0,0.009107,0.010929,0.0,0.0,0.0,0.0,0.0,0.347905,0.0,0.0,0.0,0.0,0.0,0.306011,0.041894,0.0,0.0
26762169,9763284,1421843,0,2,35.0,null,false,2,null,false,5,118,336,629,340,4,7,0.35119,0.187599,0.652427,false,142,0.9862,"""Negative""",2,8,1,219,4,66034,336785,4.619203e6,0.196072,"""article_default""",0,2,…,3,18,0.748201,0.928058,0.942446,0.28777,0.143885,0.136691,0.387265,false,8,0.94964,0.0,0.158273,0.086331,0.258993,0.0,0.0,0.0,0.0,0.014388,0.014388,0.0,0.0,0.0,0.0,0.0,0.309353,0.0,0.0,0.0,0.0,0.0,0.129496,0.028777,0.0,0.0


In [42]:
NORMALIZE_OVER_IMPRESSION_ID = [
    'trendiness_score_3d','trendiness_score_5d', 'endorsement_10h', 
    'total_pageviews/inviews', 'mean_JS','mean_topic_model_cosine', 'topics_cosine',
    'article_delay_hours', 'total_pageviews','total_inviews', 'trendiness_score_category',
    'std_JS', 'total_read_time', 'endorsement_10h_leak',"trendiness_score_3d_leak",
    "clicked_count","inview_count",
]
RANK_IMPRESSION_DESCENDING = [
    'trendiness_score_3d','trendiness_score_5d', 'endorsement_10h', 
    'total_pageviews/inviews', 'mean_JS', 'mean_topic_model_cosine', 'topics_cosine',
    'total_pageviews', 'total_read_time','total_inviews', 'trendiness_score_category',
    'endorsement_10h_leak',"trendiness_score_3d_leak","clicked_count","inview_count"
]
RANK_IMPRESSION_ASCENDING = [
    'article_delay_hours', 'std_JS', 'mean_topics_mean_delay_hours'
]
NORMALIZE_OVER_USER_ID = [
    'mean_JS', 'std_JS', 'mean_topic_model_cosine', 'topics_cosine',
]
NORMALIZE_OVER_ARTICLE = [
    'article_delay_hours', 'mean_JS', 'std_JS', 'mean_topic_model_cosine', 'topics_cosine',
]
LIST_DIVERSITY = [
    'category', 'sentiment_label', 'article_type'
]
NORMALIZE_OVER_ARTICLE_AND_USER_ID= ['endorsement_20h_articleuser']

expressions = [
    [(pl.col(c) / pl.col(c).max().over(pl.col('impression_id'))).alias(f'{c}_l_inf_impression')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).rank(method='min').over(pl.col('impression_id')).alias(f'{c}_rank_impression')
     for c in RANK_IMPRESSION_ASCENDING],
    [pl.col(c).rank(method='min', descending=True).over(pl.col('impression_id')).alias(f'{c}_rank_impression')
     for c in RANK_IMPRESSION_DESCENDING],
    [pl.col(c).std().over(pl.col('impression_id')).alias(f'std_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).skew().over(pl.col('impression_id')).alias(f'skew_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).kurtosis().over(pl.col('impression_id')).alias(f'kurtosis_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).entropy().over(pl.col('impression_id')).alias(f'entropy_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [(pl.col(c) - pl.col(c).median().over(pl.col('impression_id'))).alias(f'{c}_minus_median_impression')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [(pl.col(c) / pl.col(c).max().over(pl.col('user_id'))).alias(f'{c}_l_inf_user')
     for c in NORMALIZE_OVER_USER_ID],
    [(pl.col(c) / pl.col(c).max().over(pl.col('article'))).alias(f'{c}_l_inf_article')
     for c in NORMALIZE_OVER_ARTICLE],
    [pl.col(c).n_unique().over('impression_id').alias(f'{c}_diversity_impression')
     for c in LIST_DIVERSITY],
    [(pl.col(c) / pl.col(c).max().over([pl.col('article'),pl.col('user_id')])).alias(f'{c}_l_inf_article_user')
     for c in NORMALIZE_OVER_ARTICLE_AND_USER_ID],
]
    
for expressions_group in tqdm.tqdm(expressions):
    train_ds = train_ds.with_columns(expressions_group)
    train_ds = reduce_polars_df_memory_size(train_ds)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)

  0%|          | 0/12 [00:00<?, ?it/s]

Memory usage of dataframe is 1595.64 MB


  8%|▊         | 1/12 [00:07<01:17,  7.05s/it]

Memory usage after optimization is: 1487.14 MB
Decreased by 6.8%
Memory usage of dataframe is 1517.04 MB


 17%|█▋        | 2/12 [00:10<00:51,  5.19s/it]

...

 92%|█████████▏| 11/12 [01:17<00:05,  5.01s/it]

Memory usage after optimization is: 2519.34 MB
Decreased by 0.9%
Memory usage of dataframe is 2539.07 MB


100%|██████████| 12/12 [01:20<00:00,  6.72s/it]

Memory usage after optimization is: 2529.21 MB
Decreased by 0.4%

RAM memory GB usage = 5.328


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_total_read_time,entropy_impression_endorsement_10h_leak,entropy_impression_trendiness_score_3d_leak,entropy_impression_clicked_count,entropy_impression_inview_count,trendiness_score_3d_minus_median_impression,trendiness_score_5d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_article_user
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32
68358190,9769553,1190605,0,2,21.0,null,false,2,null,false,5,102,281,386,215,4,7,0.362989,0.264249,0.545631,false,140,0.9262,"""Negative""",2,4,33,448,3,71089,436766,5.159866e6,0.162762,"""article_default""",0,1,…,2.904595,2.917025,2.841943,2.889052,2.833379,2.734238,2.784557,103.5,128.0,5.5,-0.029081,0.051056,0.105507,0.196502,-1.5,-20470.0,-61430.0,-14.5,0.001883,-4234.0,36.0,106.0,-69.0,-167.0,0.622004,0.408247,0.968332,0.693132,0.006849,0.493583,0.494834,0.901682,0.654929,6,2,1,1.0
26762169,9763284,1421843,0,2,35.0,null,false,2,null,false,5,118,336,629,340,4,7,0.35119,0.187599,0.652427,false,142,0.9862,"""Negative""",2,8,1,219,4,66034,336785,4.619203e6,0.196072,"""article_default""",0,2,…,NaN,2.667698,2.575959,2.613662,2.63322,2.596032,1.981771,205.0,426.0,-8.0,0.014329,0.047756,0.000664,0.12415,-7.0,-6609.5,-164698.0,43.0,0.006811,-456667.0,-220.0,236.0,-161.0,-478.0,0.55619,0.437342,0.903629,0.65281,0.011976,0.308317,0.464763,0.858627,0.488477,6,3,2,1.0


# Training

In [43]:
# subsample dataset

behaviors_subsample = behaviors_train.select(['impression_id','user_id','article_ids_inview','article_ids_clicked']) \
    .pipe(
        sampling_strategy_wu2019, npratio=NPRATIO, shuffle=False, with_replacement=True, seed=123
    ).explode('article_ids_inview').drop('article_ids_clicked').rename({'article_ids_inview' : 'article'}) \
    .with_columns(pl.col('user_id').cast(pl.UInt32), pl.col('article').cast(pl.Int32)) \
    .select(['impression_id', 'user_id', 'article'])

train_ds = behaviors_subsample.join(train_ds, on=['impression_id','user_id','article'], how='left')
train_ds.head()

impression_id,user_id,article,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_total_read_time,entropy_impression_endorsement_10h_leak,entropy_impression_trendiness_score_3d_leak,entropy_impression_clicked_count,entropy_impression_inview_count,trendiness_score_3d_minus_median_impression,trendiness_score_5d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_article_user
u32,u32,i32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32
149474,139836,9778728,0,2,13.0,null,false,2,null,false,5,150,521,836,419,3,7,0.287908,0.179426,0.880068,false,142,0.9654,"""Negative""",1,5,18,251,7,22415,220247,1.004828e6,0.101772,"""article_default""",0,0,…,1.612282,1.739272,1.558951,1.774823,1.725289,-0.0,1.76343,281.5,447.5,-873.5,-0.111306,-0.005177,0.106855,-0.02667,-1.0,-62463.5,-165503.0,4.0,-0.033664,-4.981345e6,5.0,165.5,null,-886.0,0.447581,0.403328,0.963376,0.36119,0.0,0.233531,0.335164,0.891436,0.301919,4,3,1,1.0
149474,139836,9778669,0,2,13.0,null,false,2,null,false,5,85,199,313,266,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,…,1.612282,1.739272,1.558951,1.774823,1.725289,-0.0,1.76343,-40.5,-75.5,179.5,-0.013631,0.003967,-0.073142,0.015762,0.0,-10387.5,-12262.0,0.0,0.028015,-1.620564e6,-397.0,12.5,null,-56.0,0.501163,0.632488,0.750329,0.437299,0.043478,0.266627,0.511387,0.782392,0.319083,4,3,1,1.0
149474,139836,9778657,1,2,13.0,null,false,2,null,false,5,45,117,183,138,3,7,0.384615,0.245902,0.197635,false,118,0.8347,"""Neutral""",2,6,31,336,3,108389,478098,7.606737e6,0.226709,"""article_default""",0,1,…,1.612282,1.739272,1.558951,1.774823,1.725289,-0.0,1.76343,-122.5,-205.5,368.5,0.013631,0.05309,-0.001632,0.218611,0.0,23510.5,92348.0,0.0,0.051035,1.620564e6,-5.0,-115.5,null,276.0,0.78901,0.718014,0.834969,0.801144,0.066667,0.350947,0.730242,0.85965,0.604487,4,3,1,1.0
150528,143471,9778682,0,2,25.0,null,false,2,null,false,5,69,206,334,201,3,7,0.334951,0.206587,0.347973,false,498,0.9546,"""Negative""",1,5,20,267,3,143520,455723,9.298546e6,0.314928,"""article_default""",0,1,…,1.924178,2.126514,2.008478,2.181736,2.013598,-0.0,2.173429,7.0,21.0,204.0,0.088219,0.013219,0.068738,-0.001332,1.0,48254.0,82235.0,-47.0,0.013249,4.223417e6,160.0,0.0,null,498.0

In [44]:
train_ds = train_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()

categorical_columns = ['device_type', 'is_sso_user', 'gender', 'is_subscriber', 'weekday',
                       'premium', 'category', 'sentiment_label', 'is_new_article', 'is_already_seen_article',
                       'MostFrequentCategory', 'MostFrequentWeekday', 'IsFavouriteCategory',
                       'article_type', 'postcode']
categorical_columns += [f'Entity_{entity}_Present' for entity in unique_entities]
train_ds[categorical_columns] = train_ds[categorical_columns].astype('category')

X = train_ds.drop(columns=['target'])
y = train_ds['target']

In [45]:
del train_ds

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 7.444


In [46]:
def create_wide_and_deep_model(numeric_input_shape, n_layers=1, start_units=128, dropout_rate=0.1, 
                               learning_rate=1e-3, l2_lambda=1e-4, activation='relu', seed=seed,
                               l1_lambda=1e-4, weight_decay=1e-3, units_decay=2, add_gaussian_noise=False, gaussian_noise_std=0.01,
                               max_categorical_embedding=50, use_categorical_embeddings=True,
                               vocabulary_sizes: dict[str, int] = None, categorical_input_shape=None):
    
    tf.random.set_seed(seed)
    
    numeric_input_layer = tfkl.Input(shape=(numeric_input_shape,), name='NumericInput')
    
    if add_gaussian_noise:
        noise_layer = tfkl.GaussianNoise(0.01, name='NumericInputNoise')(numeric_input_layer)
        prev_layer = noise_layer
    else:
        prev_layer = numeric_input_layer
        
    # CATEGORICAL EMBEDDINGS
    if use_categorical_embeddings:
        embedding_layers = []
        categorical_inputs = []
        for feature in categorical_columns:
            vocabulary_size = vocabulary_sizes[feature]
            embedding_dim = min(max_categorical_embedding, (vocabulary_size + 1) // 2)
            cat_input = tfkl.Input(shape=(1,), name=f'{feature}Input')
            categorical_inputs.append(cat_input)
            embedding_layer = tfkl.Embedding(input_dim=vocabulary_size + 1, output_dim=embedding_dim)(cat_input)
            flatten_layer = tfkl.Flatten()(embedding_layer)
            embedding_layers.append(flatten_layer)
        wide = tfkl.Concatenate()([prev_layer] + embedding_layers)
        
    else:
        categorical_input = tfkl.Input(shape=(categorical_input_shape,), name='CategoricalInput')
        wide = tfkl.Concatenate()([prev_layer, categorical_input])
        categorical_inputs = [categorical_input]

    deep = wide
    units = start_units
    for i in range(n_layers):
        deep = tfkl.Dense(
            units=units,
            kernel_initializer=tfk.initializers.HeNormal(seed=seed),
            kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_lambda, l2=l2_lambda),
            name=f'DeepDense{i}'
        )(deep)
        deep = tfkl.BatchNormalization(name=f'DeepBatchNormalization{i}')(deep)
        deep = tfkl.Activation(activation, name=f'DeepActivation{i}')(deep)
        deep = tfkl.Dropout(dropout_rate, name=f'DeepDropout{i}')(deep)
        units = int(units / units_decay)

    merged = tfkl.Concatenate(name='WideDeepConcat')([wide, deep])
    outputs = tfkl.Dense(
        units=1,
        kernel_initializer=tfk.initializers.GlorotUniform(seed=seed),
        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_lambda, l2=l2_lambda),
        name='OutputDense',
        activation='sigmoid'
    )(merged)
    model = tfk.Model(inputs=[numeric_input_layer] + categorical_inputs, outputs=outputs)

    opt = tfk.optimizers.AdamW(learning_rate, weight_decay=weight_decay)
    loss = tfk.losses.BinaryCrossentropy()
    metrics = [tfk.metrics.AUC(curve='ROC', name='auc')]
    model.compile(loss=loss, optimizer=opt, metrics=metrics)
    
    return model

In [47]:
numerical_columns = [c for c in X.columns if c not in categorical_columns]
X_numerical = X[numerical_columns]
X_categorical = X[categorical_columns].astype(str)

categories = []
vocabulary_sizes = {}
for cat_col in categorical_columns:
    categories_train = list(X_categorical[cat_col].unique())
    categories_train.append('Unknown')
    vocabulary_sizes[cat_col] = len(categories_train)
    categories.append(categories_train)
    
categorical_encoder = OrdinalEncoder(categories=categories)
numerical_transformer = QuantileTransformer(output_distribution='normal')
    
X_categorical = categorical_encoder.fit_transform(X_categorical, y)
X_numerical = numerical_transformer.fit_transform(X_numerical.fillna(0).replace([-np.inf, np.inf], 0), y)

In [48]:
model = create_wide_and_deep_model(
    numeric_input_shape=X_numerical.shape[1], 
    categorical_input_shape=X_categorical.shape[1],
    use_categorical_embeddings=True,
    max_categorical_embedding=50,
    vocabulary_sizes=vocabulary_sizes, 
    add_gaussian_noise=True,
    gaussian_noise_std=0.01,
    n_layers=4,
    start_units=512,
    units_decay=2, # -> hidden units will be [512, 256, 128, 64]
    l1_lambda=1e-4,
    l2_lambda=1e-4,
    activation='relu',
    dropout_rate=0.2,
    learning_rate=1e-3,
    weight_decay=5e-5,
    seed=seed,
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ device_typeInput    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ is_sso_userInput    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ genderInput         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ is_subscriberInput  │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ weekdayInput        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ premiumInput        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ categoryInput       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sentiment_labelInp… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ is_new_articleInput │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ is_already_seen_ar… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MostFrequentCatego… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MostFrequentWeekda… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ IsFavouriteCategor… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ article_typeInput   │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ postcodeInput       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Entity_LOC_Present… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Entity_PER_Present… │ (None, 1)         │          0 │ -               

 Total params: 349,262 (1.33 MB)

 Trainable params: 347,342 (1.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [49]:
def scheduler(epoch, lr):
    if epoch < 2 or lr <= 1e-5:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

model.fit(
    [X_numerical] + [X_categorical[:, i].reshape(-1, 1) for i in range(X_categorical.shape[1])],
    y,
    batch_size=256,
    epochs=20,
    callbacks=[
        tfk.callbacks.LearningRateScheduler(scheduler),
    ],
)

Epoch 1/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 76s 24ms/step - auc: 0.7606 - loss: 1.2351 - learning_rate: 0.0010
Epoch 2/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 86s 31ms/step - auc: 0.8014 - loss: 0.5832 - learning_rate: 0.0010
Epoch 3/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 67s 24ms/step - auc: 0.8086 - loss: 0.5472 - learning_rate: 9.0484e-04
Epoch 4/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 82s 24ms/step - auc: 0.8140 - loss: 0.5221 - learning_rate: 8.1873e-04
Epoch 5/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 67s 24ms/step - auc: 0.8181 - loss: 0.5125 - learning_rate: 7.4082e-04
Epoch 6/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 82s 24ms/step - auc: 0.8209 - loss: 0.5080 - learning_rate: 6.7032e-04
Epoch 7/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 82s 24ms/step - auc: 0.8231 - loss: 0.5049 - learning_rate: 6.0653e-04
Epoch 8/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 67s 24ms/step - auc: 0.8248 - loss: 0.5022 - learning_rate: 5.4881e-04
Epoch 9/20
2746/2746 ━━━━━━━━━━━━━━━━━━━━ 82s 24ms/step - auc: 0.8271 - loss: 0.4991 - learning_rate: 4.

In [50]:
del history_train
del behaviors_train
del X_numerical
del X_categorical
del X

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 6.491


# Model Evaluation

In [51]:
user_id_mapping = history_val.sort('user_id').with_row_index() \
    .select(['index', 'user_id']).rename({'index': 'user_index'})

ner_interactions = history_val.select(['user_id', 'article_id_fixed']).explode('article_id_fixed') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_id_fixed', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_id_fixed']) \
    .unique(['user_index', 'ner_index'])

URM_ner = sps.csr_matrix((np.ones(ner_interactions.shape[0]),
                          (ner_interactions['user_index'].to_numpy(), ner_interactions['ner_index'].to_numpy())),
                         shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

ner_interactions_test = behaviors_val.select(['user_id', 'article_ids_clicked']).explode('article_ids_clicked') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_clicked', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_clicked']) \
    .unique(['user_index', 'ner_index'])

URM_ner_test = sps.csr_matrix((np.ones(ner_interactions_test.shape[0]),
                               (ner_interactions_test['user_index'].to_numpy(), ner_interactions_test['ner_index'].to_numpy())),
                              shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

In [52]:
item_knn = ItemKNNCFRecommender(URM_ner)
item_knn.fit(topK=100, similarity='jaccard')

evaluator = EvaluatorHoldout(URM_ner_test, cutoff_list=[10], exclude_seen=False)
result_df, _ = evaluator.evaluateRecommender(item_knn)
result_df

ItemKNNCFRecommender: URM Detected 3 ( 0.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 19035 (43.5%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 43774 (100.0%), 149.53 column/sec. Elapsed time 4.88 min
EvaluatorHoldout: Ignoring 32 ( 0.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 15310 (100.0%) in 45.97 sec. Users per second: 333


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.338445,0.339461,0.033197,0.241643,0.241977,0.493913,0.346767,0.060464,0.784651,1.033204,...,0.997914,0.783014,0.997914,0.002568,5.473111,0.953841,0.020621,0.428217,3.43338,0.010093


In [53]:
pure_svd = PureSVDRecommender(URM_ner)
pure_svd.fit(num_factors=20)

result_df, _ = evaluator.evaluateRecommender(pure_svd)
result_df

PureSVDRecommender: URM Detected 3 ( 0.0%) users with no interactions.
PureSVDRecommender: URM Detected 19035 (43.5%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 4.89 sec
EvaluatorHoldout: Processed 15310 (100.0%) in 25.94 sec. Users per second: 590


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.365343,0.368494,0.050757,0.284108,0.286194,0.69616,0.416042,0.089131,0.874853,1.344843,...,0.997914,0.873028,0.997914,0.00152,5.732489,0.955104,0.012206,0.448511,3.535079,0.010003


In [54]:
%%time

def get_inview_scores(user_ners_df, recommender, f_name):
    user_index = user_ners_df['user_index'].to_list()[0]
    ners = user_ners_df['ner_index'].to_numpy()
    scores = recommender._compute_item_score([user_index], ners)[0, ners]
    return user_ners_df.with_columns(
        pl.Series(scores).alias(f_name)
    )

ner_inview_scores = behaviors_val.select(['user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_inview', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_inview']) \
    .unique(['user_index', 'ner_index']) \
    .sort(['user_index', 'ner_index']).group_by('user_index').map_groups(
        lambda user_df: user_df.pipe(get_inview_scores, recommender=pure_svd, f_name='pure_svd_scores') \
            .pipe(get_inview_scores, recommender=item_knn, f_name='item_knn_scores')
    )

behaviors_ner_scores = behaviors_val.select(['impression_id', 'user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(ner_inview_scores, on=['user_id', 'article_ids_inview'], how='left') \
    .with_columns(pl.col(['pure_svd_scores', 'item_knn_scores']).fill_null(0)) \
    .group_by(['impression_id', 'article_ids_inview']).agg(
        pl.col('pure_svd_scores').sum().alias('total_ner_svd_scores'),
        pl.col('pure_svd_scores').mean().alias('mean_ner_svd_scores'),
        pl.col('pure_svd_scores').max().alias('max_ner_svd_scores'),
        pl.col('item_knn_scores').sum().alias('total_ner_item_knn_scores'),
        pl.col('item_knn_scores').mean().alias('mean_ner_item_knn_scores'),
        pl.col('item_knn_scores').max().alias('max_ner_item_knn_scores'),
    ).with_columns(
        pl.col(['total_ner_svd_scores', 'mean_ner_svd_scores', 
                'total_ner_item_knn_scores', 'mean_ner_item_knn_scores',
                'max_ner_svd_scores', 'max_ner_item_knn_scores']).sum().over('impression_id').name.suffix('_max')
    ).with_columns(
        # l_inf normalization
        pl.col('total_ner_svd_scores').truediv(pl.col('total_ner_svd_scores_max')),
        pl.col('mean_ner_svd_scores').truediv(pl.col('mean_ner_svd_scores_max')),
        pl.col('max_ner_svd_scores').truediv(pl.col('max_ner_svd_scores_max')),
        pl.col('total_ner_item_knn_scores').truediv(pl.col('total_ner_item_knn_scores_max')),
        pl.col('mean_ner_item_knn_scores').truediv(pl.col('mean_ner_item_knn_scores_max')),
        pl.col('max_ner_item_knn_scores').truediv(pl.col('max_ner_item_knn_scores_max')),
    ).drop(['total_ner_svd_scores_max', 'mean_ner_svd_scores_max', 
            'max_ner_svd_scores_max', 'total_ner_item_knn_scores_max',
            'mean_ner_item_knn_scores_max', 'max_ner_item_knn_scores_max']) \
    .rename({'article_ids_inview': 'article'})
behaviors_ner_scores.head(5)

CPU times: user 5min 33s, sys: 3min 49s, total: 9min 23s
Wall time: 2min 37s


impression_id,article,total_ner_svd_scores,mean_ner_svd_scores,max_ner_svd_scores,total_ner_item_knn_scores,mean_ner_item_knn_scores,max_ner_item_knn_scores
u32,i32,f32,f32,f32,f32,f32,f32
218505698,9789922,0.230195,0.366581,0.152005,0.248975,0.386579,0.171544
353071681,9695098,0.057515,0.042464,0.028477,0.032044,0.022254,0.012279
42784102,9788251,0.001599,0.002964,0.003846,0.001859,0.003567,0.005018
329803139,9779122,0.065523,0.125788,0.196605,0.04012,0.079303,0.119641
334364463,9788411,0.001463,0.001063,0.002336,0.0,0.0,0.0


In [55]:
%%time

STRONG_THR = 0.8

history_val = pl.concat(
    rows.with_columns(pl.col('article_id_fixed').list.len().alias('NumArticlesHistory')) \
        .explode(['article_id_fixed', 'impression_time_fixed', 'read_time_fixed', 'scroll_percentage_fixed']) \
        .sort(by=['user_id', 'impression_time_fixed']) \
        .with_columns(
            pl.col('impression_time_fixed').dt.weekday().alias('weekday'),
            pl.col('impression_time_fixed').dt.hour().alias('hour'),
        ).join(articles.select(['article_id', 'category', 'is_article_default', 'sentiment_label_int', 
                                'sentiment_score', 'entity_groups', 'topics']), 
               left_on='article_id_fixed', right_on='article_id', how='left') \
        .with_columns(
            (pl.col('sentiment_label_int') == 0).alias('is_neutral'),
            (pl.col('sentiment_label_int') == 1).alias('is_positive'),
            (pl.col('sentiment_label_int') == -1).alias('is_negative'),
            ((pl.col('sentiment_label_int') == 0) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_neutral'),
            ((pl.col('sentiment_label_int') == 1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_positive'),
            ((pl.col('sentiment_label_int') == -1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_negative'),
            pl.col('entity_groups').list.unique(),
        ).group_by('user_id').agg(
            pl.col('article_id_fixed'),
            pl.col('impression_time_fixed'),
            pl.col('category'),
            pl.col('NumArticlesHistory').first(),
            pl.col('read_time_fixed').median().alias('MedianReadTime'),
            pl.col('read_time_fixed').max().alias('MaxReadTime'),
            pl.col('read_time_fixed').sum().alias('TotalReadTime'),
            pl.col('scroll_percentage_fixed').median().alias('MedianScrollPercentage'),
            pl.col('scroll_percentage_fixed').max().alias('MaxScrollPercentage'),
            (pl.col('is_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('NeutralPct'),
            (pl.col('is_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PositivePct'),
            (pl.col('is_negative').sum() / pl.col('NumArticlesHistory').first()).alias('NegativePct'),
            (pl.col('strong_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNeutral'),
            (pl.col('strong_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongPositive'),
            (pl.col('strong_negative').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNegative'),
            (1 - (pl.col('is_article_default').sum() / pl.col('NumArticlesHistory').first())).alias('PctNotDefaultArticles'),
            pl.col('category').mode().alias('MostFrequentCategory'),
            pl.col('weekday').mode().alias('MostFrequentWeekday'),
            pl.col('hour').mode().alias('MostFrequentHour'),
            pl.col('entity_groups').flatten(),
            pl.col('topics').flatten().alias('topics_flatten')
        ).pipe(add_topics_tf_idf_columns, topics_col='topics_flatten', vectorizer=vectorizer) \
        .drop('topics_flatten').with_columns(
            pl.col('MostFrequentCategory').list.first(),
            pl.col('MostFrequentWeekday').list.first(),
            pl.col('MostFrequentHour').list.first(),
        ).with_columns(
            (pl.col('entity_groups').list.count_matches(entity) / pl.col('NumArticlesHistory')).alias(f'{entity}Pct')
            for entity in unique_entities
        ).drop('entity_groups')
    for rows in tqdm.tqdm(history_val.iter_slices(1000), total=history_val.shape[0] // 1000)
)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
history_val = reduce_polars_df_memory_size(history_val)
history_val.head(2)

16it [00:16,  1.02s/it]                        



RAM memory GB usage = 7.145
Memory usage of dataframe is 36.51 MB
Memory usage after optimization is: 35.72 MB
Decreased by 2.2%
CPU times: user 19.7 s, sys: 1.37 s, total: 21.1 s
Wall time: 17.2 s


user_id,article_id_fixed,impression_time_fixed,category,NumArticlesHistory,MedianReadTime,MaxReadTime,TotalReadTime,MedianScrollPercentage,MaxScrollPercentage,NeutralPct,PositivePct,NegativePct,PctStrongNeutral,PctStrongPositive,PctStrongNegative,PctNotDefaultArticles,MostFrequentCategory,MostFrequentWeekday,MostFrequentHour,topics_flatten_tf_idf,LOCPct,PERPct,ORGPct,PRODPct,MISCPct,EVENTPct
u32,list[i32],list[datetime[μs]],list[i16],u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i8,i8,list[f32],f32,f32,f32,f32,f32,f32
10623,"[9750749, 9685790, … 9779517]","[2023-05-05 07:51:01, 2023-05-05 07:51:36, … 2023-05-24 17:10:11]","[118, 457, … 414]",150,3.5,1563.0,9330.0,37.0,100.0,0.28,0.093333,0.626667,0.126667,0.06,0.546667,0.06,118,1,20,"[0.0, 0.042172, … 0.089681]",0.846667,0.893333,0.913333,0.213333,0.266667,0.3
12718,"[9749036, 9749025, … 9778718]","[2023-05-04 07:49:49, 2023-05-04 07:50:15, … 2023-05-24 07:07:25]","[118, 118, … 140]",114,7.0,748.0,3440.0,36.0,100.0,0.245614,0.052632,0.701754,0.087719,0.035088,0.614035,0.035088,118,1,7,"[0.0, 0.048524, … 0.217846]",0.815789,0.807018,0.947368,0.254386,0.192982,0.114035


In [56]:
last_history_df = history_val.with_columns(
    pl.col('impression_time_fixed').list.max().alias('last_history_impression_time'),
    pl.col('article_id_fixed').list.tail(1).alias('last_history_article'),
).select(['user_id', 'last_history_impression_time', 'last_history_article'])

last_session_time_df = behaviors_val.select(['session_id', 'user_id', 'impression_time', 'article_ids_inview', 'article_ids_clicked']) \
    .group_by('session_id').agg(
        pl.col('user_id').first(), 
        pl.col('impression_time').max().alias('session_time'), 
        pl.col('article_ids_inview').flatten().alias('all_seen_articles'),
        (pl.col('impression_time').max() - pl.col('impression_time').min()).dt.total_minutes().alias('session_duration'),
    ).with_columns(
        pl.col(['session_time', 'session_duration']).shift(1).over('user_id').name.prefix('last_'),
        pl.col('all_seen_articles').list.unique().shift(1).over('user_id'),
        pl.col('session_duration').rolling_mean(100, min_periods=1).over('user_id').alias('mean_prev_sessions_duration'),
    ).with_columns(pl.col(['last_session_duration']).fill_null(0)) \
    .join(last_history_df, on='user_id', how='left') \
    .with_columns(
        pl.col('last_session_time').fill_null(pl.col('last_history_impression_time')),
        pl.col('all_seen_articles').fill_null(pl.col('last_history_article')),
    ).select(['session_id', 'last_session_time', 'last_session_duration',
              'all_seen_articles', 'mean_prev_sessions_duration'])

In [57]:
%%time

NPRATIO = 2

users_mean_trendiness_scores, topics_mean_trendiness_scores = _preprocessing_history_trendiness_scores(
        history=history_val, articles=articles)
topic_mean_delays, user_mean_delays = _preprocessing_mean_delay_features(
    articles=articles, history=history_val)
windows, user_windows, user_topics_windows, user_category_windows = _preprocessing_window_features(
    history=history_val, articles=articles)
articles_endorsement = _preprocessing_article_endorsement_feature(
    behaviors=behaviors_val, period="10h")
normalized_endorsement = normalize_endorsement(articles_endorsement)
articles_endorsement_articleuser = _preprocessing_article_endorsement_feature_by_article_and_user(
    behaviors=behaviors_val, period="20h")
normalized_endorsement_articleuser = normalize_endorsement_by_article_and_user(articles_endorsement_articleuser)

history_counts=history_val.select(['article_id_fixed']).explode('article_id_fixed').group_by('article_id_fixed').len() \
.rename({'article_id_fixed':'article','len':'clicked_count'})

behaviors_counts=behaviors_val.select('article_ids_inview').explode('article_ids_inview').group_by('article_ids_inview').len() \
.rename({'article_ids_inview':'article','len':'inview_count'})

# dropping duplicates that can occur because of sampling strategy
val_ds = behaviors_val.pipe(create_binary_labels_column, shuffle=True, seed=123) \
    .with_columns(pl.col('gender').fill_null(2)) \
    .with_columns(pl.col('postcode').fill_null(5)) \
    .select(['impression_id', 'article_ids_inview', 'impression_time', 'labels', 
             'device_type', 'read_time', 'scroll_percentage', 'user_id', 'is_sso_user', 'gender',
             'age', 'is_subscriber', 'session_id', 'postcode']) \
    .explode(['article_ids_inview', 'labels']) \
    .rename({'article_ids_inview': 'article', 'labels': 'target'}) \
    .with_columns(pl.col('article').cast(pl.Int32)) \
    .pipe(add_trendiness_feature, articles=articles, period='1d') \
    .rename({'trendiness_score': 'trendiness_score_1d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='3d') \
    .rename({'trendiness_score': 'trendiness_score_3d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='5d') \
    .rename({'trendiness_score': 'trendiness_score_5d'}) \
    .pipe(add_trendiness_feature_leak, articles=articles, period='3d') \
    .rename({'trendiness_score_leak':'trendiness_score_3d_leak'}) \
    .unique(['impression_id', 'article']) \
    .with_columns(
        pl.col('impression_time').dt.weekday().alias('weekday'),
        pl.col('impression_time').dt.hour().alias('hour'),
        pl.col('article').cast(pl.Int32),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_3d')
        ).alias('trendiness_score_1d/3d'),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_5d')
        ).alias('trendiness_score_1d/5d'),
        (
            pl.col('trendiness_score_3d') / 
            pl.col('trendiness_score_3d').max().over(pl.col('impression_time').dt.date())
        ).alias('normalized_trendiness_score_overall'),
    ).join(articles.select(['article_id', 'premium', 'published_time', 'category',
                            'sentiment_score', 'sentiment_label', 'entity_groups',
                            'num_images', 'title_len', 'subtitle_len', 'body_len','num_topics',
                            'total_pageviews', 'total_inviews', 'total_read_time', 
                            'total_pageviews/inviews', 'article_type']),
           left_on='article', right_on='article_id', how='left') \
    .with_columns(
        (pl.col('impression_time') - pl.col('published_time')).dt.total_days().alias('article_delay_days'),
        (pl.col('impression_time') - pl.col('published_time')).dt.total_hours().alias('article_delay_hours')
    ).with_columns(
        pl.col('entity_groups').list.contains(entity).alias(f'Entity_{entity}_Present')
        for entity in unique_entities
    ).drop('entity_groups') \
    .join(behaviors_ner_scores, on=['impression_id', 'article'], how='left') \
    .pipe(add_history_trendiness_scores_feature, articles=articles, 
          users_mean_trendiness_scores=users_mean_trendiness_scores,
          topics_mean_trendiness_scores=topics_mean_trendiness_scores, topics=topics) \
    .pipe(add_mean_delays_features, articles=articles,
          topic_mean_delays=topic_mean_delays, user_mean_delays=user_mean_delays) \
    .pipe(add_window_features, articles=articles, user_windows=user_windows,
          user_category_windows=user_category_windows, 
          user_topics_windows=user_topics_windows, windows=windows) \
    .pipe(add_trendiness_feature_categories, articles=articles) \
    .pipe(add_article_endorsement_feature, normalized_endorsement=normalized_endorsement) \
    .pipe(add_article_endorsement_feature_by_article_and_user, normalized_endorsement = normalized_endorsement_articleuser) \
    .pipe(add_article_endorsement_feature_leak, normalized_endorsement = normalized_endorsement) \
    .rename({
        "endorsement_10h_diff_rolling_right":"endorsement_10h_leak_diff_rolling",
        "endorsement_macd_right":"endorsement_leak_macd",
        "endorsement_quantile_norm_10h_right":"endorsement_quantile_norm_10h_leak",
        "normalized_endorsement_10h_rolling_max_ratio_right":"normalized_endorsement_10h_leak_rolling_max_ratio"
    }).join(history_counts, on='article',how='left') \
    .join(behaviors_counts , on='article', how='left') \
    .join(last_session_time_df, on='session_id', how='left').with_columns(
        (pl.col('impression_time') - pl.col('last_session_time')).dt.total_hours().alias('last_session_time_hour_diff'),
        ((pl.col('last_session_time') - pl.col('published_time')).dt.total_hours() > 0).alias('is_new_article'),
        pl.col('all_seen_articles').list.contains(pl.col('article')).alias('is_already_seen_article'),
    ).drop(['published_time', 'session_id', 'all_seen_articles', 'last_session_time']) \
    .join(published_category_popularity, how='left', right_on=['published_date', 'category'],
          left_on=[pl.col('impression_time').dt.date() - pl.duration(days=1), 'category']) \
    .rename({'category_daily_pct': 'yesterday_category_daily_pct'}) \
    .with_columns(pl.col('yesterday_category_daily_pct').fill_null(0))

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)

val_ds = reduce_polars_df_memory_size(val_ds)
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
val_ds.head()

16it [00:25,  1.58s/it]
2205it [00:06, 329.89it/s]                          
16it [00:01,  8.77it/s]                        
16it [00:02,  7.78it/s]                        
100%|██████████| 507/507 [00:06<00:00, 73.40it/s]



RAM memory GB usage = 8.889
Memory usage of dataframe is 1314.28 MB
Memory usage after optimization is: 735.73 MB
Decreased by 44.0%

RAM memory GB usage = 8.285
CPU times: user 14min 2s, sys: 1min 13s, total: 15min 15s
Wall time: 4min 37s


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,…,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article,yesterday_category_daily_pct
u32,i32,u32,date,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,…,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16,i8,f32,i16,bool,bool,f32
311999725,9779956,2054298,2023-05-24,0,2,80.0,null,false,2,null,false,5,54,151,222,141,4,7,0.357616,0.243243,0.276557,false,118,0.9876,"""Negative""",1,10,17,326,4,20536,231786,2.192031e6,0.088599,"""article_default""",0,…,99,40,17,11,1,0,0,0,35,45,109,3,0.019231,0.0,0.0,1.0,0.5,1,0.058824,0.0,0.0,1.0,1.0,374,0.003601,5.1,2.9,0.204595,0.147651,247,566,1,2.428571,-146,true,false,0.239437
349213169,9779564,1361500,2023-05-24,0,2,15.0,null,false,2,null,false,5,130,394,624,333,4,7,0.329949,0.208333,0.721612,false,142,0.9832,"""Negative""",2,8,18,240,4,13049,202586,1.120723e6,0.064412,"""article_default""",0,…,270,174,76,99,1,0,0,0,64,239,112,7,0.044872,0.0,0.0,2.333333,1.0,1,0.166667,0.0,0.0,1.0,1.0,1210,0.01051,-263.700012,-27.299999,0.621469,0.524718,56,1580,0,0.0,-57,true,false,0.239437
311999725,9778939,2054298,2023-05-24,0,2,80.0,null,false,2,null,false,5,110,319,484,255,4,7,0.344828,0.227273,0.584249,true,414,0.9926,"""Negative""",3,8,18,776,5,144704,1279706,8.283967e6,0.113076,"""article_default""",0,…,99,40,17,11,1,0,0,0,30,88,58,2,0.012821,0.0,0.0,0.666667,0.25,1,0.058824,0.0,0.0,1.0,1.0,5058,0.050717,10.4,5.8,2.690426,1.0,1312,7220,1,2.428571,-146,true,false,0.147887
118493914,9776967,1770038,2023-05-24,0,1,840.0,100.0,false,2,null,false,5,112,317,474,270,4,7,0.353312,0.236287,0.580586,false,118,0.9929,"""Negative""",2,7,28,236,5,54155,496389,2.747845e6,0.109098,"""article_default""",0,…,88,45,43,13,1,0,0,0,52,113,109,1,0.00641,0.0,0.0,0.333333,0.333333,1,0.027027,0.0,0.0,1.0,1.0,453,0.006062,-34.200001,-1.8,0.237173,0.226613,743,834,0,4.5,-145,true,false,0.239437
565885711,9779263,1389619,2023-05-24,0,1,27.0,null,true,0,null,false,5,60,180,291,195,4,7,0.333333,0.206186,0.32967,false,498,0.5646,"""Positive""",3,6,13,501,4,197659,771668,1.5790282e7,0.256145,"""article_default""",0,…,393,281,104,241,1,0,0,0,7,91,8,8,0.051282,0.0,0.0,2.666667,1.0,1,0.058824,0.0,0.0,1.0,1.0,1698,0.014553,-191.699997,-19.299999,0.960407,1.0,2243,2714,0,0.166667,-41,true,false,0.028169


In [58]:
val_ds.shape

(2928942, 93)

In [59]:
prev_columns = [c for c in val_ds.columns if c not in ['impression_id', 'article', 'impression_time']]

val_ds = pl.concat(
    rows.join(history_val.select(['user_id', 'article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .join(articles.select(['article_id', 'topics', 'entity_groups', 'topics_idf'] + topic_model_columns), 
              left_on='article', right_on='article_id', how='left') \
        .explode(['article_id_fixed', 'impression_time_fixed']) \
        .join(articles.select(['article_id', 'topics', 'entity_groups'] + topic_model_columns), 
              left_on='article_id_fixed', right_on='article_id', how='left') \
        .rename({'topics_right': 'topics_history', 'entity_groups_right': 'entity_groups_history'}) \
        .rename({f'topic_model_embedding_{i}_right': f'topic_model_embedding_{i}_history' for i in range(N_COMPONENTS)}) \
        .with_columns(
            # 1/delay gives the weight for the weighted mean of the lda embeddings
            (1 / (pl.col('impression_time') - pl.col('impression_time_fixed')).dt.total_hours().sqrt()).alias('history_weight'),
            (pl.col("topics").list.set_intersection(pl.col("topics_history")).list.len().truediv(
                pl.col("topics").list.set_union(pl.col("topics_history")).list.len()
            )).alias("JS"),
            pl.col('entity_groups').list.set_intersection(pl.col('entity_groups_history')).list.len().alias('common_entities'),
        ).drop(['entity_groups_history', 'entity_groups', 'topics', 'topics_history']) \
        .with_columns(
            # summing delays to normalize them before weighted mean
            pl.col('history_weight').sum().over(['impression_id', 'article']).alias('history_weight_sum'),
            *[pl.col(x).mul(pl.col(f'{x}_history')).alias(f'{x}_dot') for x in topic_model_columns],
            *[pl.col(x).mul(pl.col(x)) for x in topic_model_columns],
            *[pl.col(f'{x}_history').mul(pl.col(f'{x}_history')).alias(f'{x}_history_square') for x in topic_model_columns],
        ).with_columns(
            # weights now sum to 1
            pl.col('history_weight').truediv(pl.col('history_weight_sum')),
            pl.sum_horizontal(topic_model_columns).sqrt().alias('topic_model_norm'),
            pl.sum_horizontal([f'{x}_history_square' for x in topic_model_columns]).sqrt().alias('topic_model_history_norm'),
            pl.sum_horizontal([f'{x}_dot' for x in topic_model_columns]).alias('topic_model_dot'),
        ).with_columns(
            *[pl.col(f'{x}_history').mul(pl.col('history_weight')).alias(f'{x}_history_weighted') for i, x in enumerate(topic_model_columns)],
            pl.col('topic_model_dot').truediv(pl.col('topic_model_norm').mul('topic_model_history_norm')).alias('topic_model_cosine')
        ).group_by(['impression_id', 'article']).agg(
            pl.col(prev_train_columns).first(),
            pl.col('topics_idf').first(),
            pl.col('common_entities').mean().alias('MeanCommonEntities'),
            pl.col('common_entities').max().alias('MaxCommonEntities'),
            pl.col("JS").mean().alias("mean_JS"),
            pl.col("JS").min().alias("min_JS"),
            pl.col("JS").max().alias("max_JS"),
            pl.col("JS").std().alias("std_JS"),
            pl.col("JS").mul(pl.col("history_weight")).sum().alias("weighted_mean_JS"),
            pl.col("topic_model_cosine").mean().alias("mean_topic_model_cosine"),
            pl.col("topic_model_cosine").min().alias("min_topic_model_cosine"),
            pl.col("topic_model_cosine").max().alias("max_topic_model_cosine"),
            pl.col("topic_model_cosine").std().alias("std_topic_model_cosine"),
            pl.col("topic_model_cosine").mul(pl.col("history_weight")).sum().alias("weighted_mean_topic_model_cosine"),
            *[pl.col(f'{x}_history').mean().alias(f'lda_{i}_history_mean') for i, x in enumerate(topic_model_columns)],
            *[pl.col(f'{x}_history_weighted').sum().alias(f'lda_{i}_history_weighted_mean') for i, x in enumerate(topic_model_columns)]
        ).join(history_val.drop(['article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .with_columns(
            pl.struct(['topics_idf', 'topics_flatten_tf_idf']).map_elements(
                lambda x: cosine_similarity(x['topics_idf'], x['topics_flatten_tf_idf']), return_dtype=pl.Float64
            ).cast(pl.Float32).alias('topics_cosine'),
            (pl.col('category') == pl.col('MostFrequentCategory')).alias('IsFavouriteCategory'),
            pl.col('category_right').list.n_unique().alias('NumberDifferentCategories'),
            list_pct_matches_with_col('category_right', 'category').alias('PctCategoryMatches'),
        ).with_columns(
            [list_pct_matches_with_constant('category_right', c).alias(f'Category_{c_str}_Pct')
             for c, c_str in unique_categories.items()]
        ).drop(['topics_idf', 'topics_flatten', 'topics_flatten_tf_idf', 'category_right'])
    for rows in tqdm.tqdm(val_ds.iter_slices(10000), total=val_ds.shape[0] // 10000)
)

val_ds = reduce_polars_df_memory_size(val_ds)
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
val_ds.head()

293it [38:09,  7.81s/it]


Memory usage of dataframe is 1925.91 MB
Memory usage after optimization is: 1492.96 MB
Decreased by 22.5%

RAM memory GB usage = 10.47


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,MostFrequentWeekday,MostFrequentHour,LOCPct,PERPct,ORGPct,PRODPct,MISCPct,EVENTPct,topics_cosine,IsFavouriteCategory,NumberDifferentCategories,PctCategoryMatches,Category_ferie_Pct,Category_underholdning_Pct,Category_penge_Pct,Category_krimi_Pct,Category_bibliotek_Pct,Category_side9_Pct,Category_podcast_Pct,Category_haandvaerkeren_Pct,Category_forbrug_Pct,Category_musik_Pct,Category_horoskoper_Pct,Category_auto_Pct,Category_dagsorden_Pct,Category_plus_Pct,Category_vin_Pct,Category_nyheder_Pct,Category_om_ekstra_bladet_Pct,Category_biler_Pct,Category_opinionen_Pct,Category_services_Pct,Category_incoming_Pct,Category_sport_Pct,Category_nationen_Pct,Category_video_Pct,Category_sex_og_samliv_Pct
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,i8,i8,f32,f32,f32,f32,f32,f32,f32,bool,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
18624038,9778971,1175611,1,2,24.0,null,false,2,null,false,5,118,360,584,299,4,7,0.327778,0.202055,0.659341,false,142,0.7911,"""Neutral""",2,9,23,256,4,104102,497489,6.707142e6,0.209255,"""article_default""",0,10,…,1,11,0.76555,0.854067,0.91866,0.27512,0.208134,0.217703,0.42605,false,8,0.983254,0.0,0.150718,0.023923,0.198565,0.0,0.0,0.0,0.0,0.021531,0.026316,0.0,0.0,0.0,0.0,0.0,0.287081,0.0,0.0,0.0,0.0,0.0,0.251196,0.04067,0.0,0.0
36672693,9777034,1895052,0,1,33.0,null,false,2,null,false,5,67,191,270,119,4,7,0.350785,0.248148,0.349817,false,414,0.8756,"""Negative""",1,8,21,336,8,51562,350267,2.754714e6,0.147208,"""article_default""",1,26,…,1,7,0.787286,0.875306,0.92665,0.242054,0.195599,0.188264,0.140307,false,8,0.982885,0.0,0.085575,0.0489,0.200489,0.0,0.0,0.0,0.0,0.01467,0.017115,0.0,0.0,0.0,0.0,0.0,0.391198,0.0,0.0,0.0,0.0,0.0,0.141809,0.100244,0.0,0.0
384352527,9647575,290548,0,2,42.0,null,false,2,null,false,5,103,309,500,250,4,7,0.333333,0.206,0.565934,true,414,0.9136,"""Neutral""",10,6,26,612,3,45301,913674,2.24602e6,0.049581,"""article_default""",27,650,…,2,8,0.792109,0.889226,0.911988,0.259484,0.251897,0.265554,0.453631,false,11,0.984825,0.001517,0.141123,0.030349,0.113809,0.0,0.003035,0.0,0.0,0.015175,0.021244,0.0,0.0,0.0,0.0,0.0,0.306525,0.0,0.0,0.0,0.0,0.0,0.323217,0.040971,0.0,0.003035
169708633,9779860,835342,0,2,152.0,100.0,false,2,null,false,5,54,151,222,141,4,7,0.357616,0.243243,0.276557,false,118,0.9862,"""Negative""",0,7,17,273,4,216059,978794,1.1436902e7,0.22074,"""article_default""",0,10,…,5,12,0.821192,0.774834,0.927152,0.15894,0.178808,0.10596,0.051616,true,8,0.953642,0.0,0.125828,0.006623,0.304636,0.0,0.0,0.0,0.0,0.02649,0.039735,0.0,0.0,0.0,0.0,0.0,0.370861,0.0,0.0,0.0,0.0,0.0,0.10596,0.019868,0.0,0.0
150263899,9779564,2445823,0,1,13.0,null,false,2,null,false,5,130,394,624,333,4,7,0.329949,0.208333,0.721612,false,142,0.9832,"""Negative""",2,8,18,240,4,13049,202586,1.120723e6,0.064412,"""article_default""",0,0,…,3,7,0.766102,0.833898,0.935593,0.298305,0.20339,0.216949,0.3895,false,8,0.976271,0.0,0.162712,0.054237,0.2,0.0,0.0,0.0,0.0,0.00339,0.027119,0.0,0.0,0.0,0.0,0.0,0.372881,0.0,0.0,0.0,0.0,0.0,0.149153,0.030508,0.0,0.0


In [60]:
for expressions_group in tqdm.tqdm(expressions):
    val_ds = val_ds.with_columns(expressions_group)
    val_ds = reduce_polars_df_memory_size(val_ds)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
val_ds.head(2)

  0%|          | 0/12 [00:00<?, ?it/s]

Memory usage of dataframe is 1807.55 MB


  8%|▊         | 1/12 [00:08<01:28,  8.01s/it]

Memory usage after optimization is: 1695.82 MB
Decreased by 6.2%
Memory usage of dataframe is 1729.69 MB


 17%|█▋        | 2/12 [00:12<00:57,  5.74s/it]

...

 92%|█████████▏| 11/12 [01:28<00:06,  6.00s/it]

Memory usage after optimization is: 2864.89 MB
Decreased by 0.9%
Memory usage of dataframe is 2887.23 MB


100%|██████████| 12/12 [01:33<00:00,  7.76s/it]

Memory usage after optimization is: 2876.06 MB
Decreased by 0.4%

RAM memory GB usage = 10.81


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_total_read_time,entropy_impression_endorsement_10h_leak,entropy_impression_trendiness_score_3d_leak,entropy_impression_clicked_count,entropy_impression_inview_count,trendiness_score_3d_minus_median_impression,trendiness_score_5d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_article_user
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32
18624038,9778971,1175611,1,2,24.0,null,false,2,null,false,5,118,360,584,299,4,7,0.327778,0.202055,0.659341,false,142,0.7911,"""Neutral""",2,9,23,256,4,104102,497489,6.707142e6,0.209255,"""article_default""",0,10,…,2.415137,2.513112,2.205569,2.370303,2.460781,2.203103,2.248056,117.0,206.0,-17.0,0.100157,0.026682,0.049333,0.051175,0.0,49947.0,1100.0,3.0,0.010692,3.959297e6,-152.0,94.0,1086.0,-218.0,0.803862,0.679927,0.927793,0.798806,0.064516,0.397469,0.554713,0.934377,0.577377,5,2,1,1.0
36672693,9777034,1895052,0,1,33.0,null,false,2,null,false,5,67,191,270,119,4,7,0.350785,0.248148,0.349817,false,414,0.8756,"""Negative""",1,8,21,336,8,51562,350267,2.754714e6,0.147208,"""article_default""",1,26,…,3.10917,3.208914,2.644696,2.683124,3.082204,2.813845,2.595687,-0.5,7.0,-3.0,0.008896,-0.040181,0.061253,-0.139116,14.0,-6897.5,-67259.5,0.5,-0.086881,-59373.0,-112.5,-34.0,196.0,-30.0,0.211709,0.196192,0.944149,0.233657,0.155689,0.138256,0.202344,0.900108,0.275149,7,3,1,1.0


In [61]:
del history_val
del articles
del behaviors_val

gc.collect()

0

In [62]:
val_ds.shape

(2928942, 298)

In [63]:
X_val = val_ds.drop(['impression_id', 'article', 'user_id', 'target']).to_pandas()
val_ds = val_ds.select(['impression_id', 'article', 'target'])

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
gc.collect()


RAM memory GB usage = 13.68


0

In [64]:
X_val[numerical_columns] = X_val[numerical_columns].astype(np.float32)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
gc.collect()

for i, cat_col in enumerate(categorical_columns):
    X_val[cat_col] = X_val[cat_col].astype(str)
    categories_val = list(X_val[cat_col].unique())
    unknown_categories = [x for x in categories_val if x not in categories[i]]
    X_val[cat_col] = X_val[cat_col].astype(str).replace(list(unknown_categories), 'Unknown')
    
X_val[categorical_columns] = categorical_encoder.transform(X_val[categorical_columns]).astype(np.float32)
X_val[numerical_columns] = numerical_transformer.transform(X_val[numerical_columns].fillna(0).replace([-np.inf, np.inf], 0)).astype(np.float32)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color=Fore.RED)


RAM memory GB usage = 16.55

RAM memory GB usage = 13.93


In [65]:
val_ds = val_ds.with_columns(
    pl.Series(
        model.predict([X_val[numerical_columns].values] + [X_val[col].values.reshape(-1, 1) for col in categorical_columns], batch_size=256).flatten()
    ).alias('prediction')
)
val_ds.select(['impression_id', 'target', 'prediction'])

11442/11442 ━━━━━━━━━━━━━━━━━━━━ 102s 9ms/step


impression_id,target,prediction
u32,i8,f32
18624038,1,0.678817
36672693,0,0.061597
384352527,0,0.34076
169708633,0,0.236041
150263899,0,0.390468
…,…,…
502501633,0,0.017104
222959401,0,0.537127
114067178,1,0.319391


In [66]:
del X_val

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color=Fore.RED)


RAM memory GB usage = 13.95


In [67]:
evaluation_ds = val_ds.group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
evaluation_ds

impression_id,target,prediction
u32,list[i8],list[f32]
274845439,"[0, 0, … 0]","[0.597403, 0.036727, … 0.210239]"
479643508,"[0, 1, … 0]","[0.389359, 0.210003, … 0.069514]"
287333920,"[1, 0, … 0]","[0.408522, 0.083474, … 0.081962]"
220871335,"[0, 0, … 0]","[0.360957, 0.024378, … 0.039176]"
98866035,"[0, 0, … 0]","[0.035912, 0.019668, … 0.039165]"
…,…,…
216859845,"[0, 0, … 0]","[0.046129, 0.312971, … 0.098609]"
120512604,"[0, 0, … 0]","[0.13756, 0.033969, … 0.028542]"
347228980,"[0, 0, … 0]","[0.648804, 0.683085, … 0.14262]"


In [68]:
%%time

met_eval = MetricEvaluator(
    labels=evaluation_ds['target'].to_list(),
    predictions=evaluation_ds['prediction'].to_list(),
    metric_functions=[
        AucScore(),
        MrrScore(),
        NdcgScore(k=5),
        NdcgScore(k=10),
    ],
)
met_eval.evaluate()

CPU times: user 8min 43s, sys: 104 ms, total: 8min 43s
Wall time: 8min 44s


<MetricEvaluator class>: 
 {
    "auc": 0.8031585969871823,
    "mrr": 0.5888105178548648,
    "ndcg@5": 0.6556428956320332,
    "ndcg@10": 0.6811316485917643
}

In [69]:
import joblib

model.save('/kaggle/working/wide_deep.keras')
joblib.dump(categorical_encoder, '/kaggle/working/categorical_encoder.joblib')
joblib.dump(numerical_transformer, '/kaggle/working/numerical_transformer.joblib')
val_ds.write_parquet('/kaggle/working/wide_deep_predictions.parquet')